# Evaluación Integral del Pipeline RAG

Esta notebook evalúa el pipeline completo de RAG (Retrieval-Augmented Generation) incluyendo:
- **Retrievers** (embeddings + Qdrant)
- **Re-ranking** de documentos
- **Generación** con LLM
- **Métricas objetivas** (recall, nDCG, MRR, precisión@k)
- **Métricas subjetivas** (coherencia, relevancia, completitud via LLM-as-a-judge)

## Tecnologías utilizadas:
- Qdrant (base vectorial)
- Sentence Transformers (embeddings)
- Cross-encoders (re-ranking)
- OpenAI/Transformers (generación y evaluación)
- Scikit-learn (métricas)


## 1. Importaciones y Configuración Inicial


In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import Dict, List, Any, Tuple, Optional
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http import models

# Embeddings
from sentence_transformers import SentenceTransformer, CrossEncoder

# Métricas
from sklearn.metrics import ndcg_score
from sklearn.preprocessing import LabelBinarizer

# LLM
import openai
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Librerías importadas correctamente")


## 2. Configuración de Modelos y Parámetros


In [ ]:
# Configuración de modelos de embedding
EMBEDDING_MODELS = {
    'all-MiniLM-L6-v2': {
        'model_name': 'sentence-transformers/all-MiniLM-L6-v2',
        'dimension': 384,
        'description': 'Modelo ligero y rápido'
    },
    'all-mpnet-base-v2': {
        'model_name': 'sentence-transformers/all-mpnet-base-v2',
        'dimension': 768,
        'description': 'Modelo balanceado'
    },
    'paraphrase-multilingual-MiniLM-L12-v2': {
        'model_name': 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2',
        'dimension': 384,
        'description': 'Modelo multilingüe'
    },
    'all-distilroberta-v1': {
        'model_name': 'sentence-transformers/all-distilroberta-v1',
        'dimension': 768,
        'description': 'Modelo basado en DistilRoBERTa'
    }
}

# Configuración de modelos de re-ranking
RERANKING_MODELS = {
    'ms-marco-MiniLM-L-6-v2': {
        'model_name': 'cross-encoder/ms-marco-MiniLM-L-6-v2',
        'description': 'Re-ranker ligero para MS MARCO'
    },
    'ms-marco-MiniLM-L-12-v2': {
        'model_name': 'cross-encoder/ms-marco-MiniLM-L-12-v2',
        'description': 'Re-ranker más robusto para MS MARCO'
    },
    'ms-marco-MiniLM-L-2-v2': {
        'model_name': 'cross-encoder/ms-marco-MiniLM-L-2-v2',
        'description': 'Re-ranker ultra-ligero'
    }
}

# Configuración de métricas de retrieval
RETRIEVAL_METRICS = {
    'recall_at_k': [1, 3, 5, 10],
    'precision_at_k': [1, 3, 5, 10],
    'ndcg_at_k': [1, 3, 5, 10],
    'mrr': True,
    'hit_rate_at_k': [1, 3, 5, 10]
}

# Configuración de evaluación LLM
LLM_EVALUATION_CRITERIA = {
    'coherence': {
        'description': '¿La respuesta es coherente y bien estructurada?',
        'scale': (1, 5)
    },
    'relevance': {
        'description': '¿La respuesta es relevante a la pregunta?',
        'scale': (1, 5)
    },
    'completeness': {
        'description': '¿La respuesta es completa y abarca todos los aspectos?',
        'scale': (1, 5)
    },
    'fidelity': {
        'description': '¿La respuesta es fiel al contexto proporcionado?',
        'scale': (1, 5)
    },
    'conciseness': {
        'description': '¿La respuesta es concisa sin ser incompleta?',
        'scale': (1, 5)
    }
}

# Configuración de Qdrant
QDRANT_CONFIG = {
    'url': 'http://localhost:6333',
    'collection_name': 'lus_laboris_articles',
    'top_k': 20  # Número de documentos a recuperar inicialmente
}

# Configuración de LLM
LLM_CONFIG = {
    'provider': 'openai',  # 'openai' o 'huggingface'
    'model': 'gpt-3.5-turbo',
    'temperature': 0.1,
    'max_tokens': 500
}

print("✅ Configuración de modelos y parámetros definida")


## 3. Carga del Dataset de Evaluación


In [ ]:
def load_evaluation_dataset(file_path: str) -> pd.DataFrame:
    """
    Carga el dataset de evaluación con preguntas y respuestas esperadas
    
    Estructura esperada:
    - question: Pregunta a evaluar
    - expected_answer: Respuesta esperada
    - expected_articles: Lista de artículos relevantes (IDs o contenido)
    - category: Categoría de la pregunta (opcional)
    - difficulty: Nivel de dificultad (opcional)
    """
    if file_path.endswith('.json'):
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return pd.DataFrame(data)
    elif file_path.endswith('.csv'):
        return pd.read_csv(file_path)
    else:
        raise ValueError("Formato de archivo no soportado. Use .json o .csv")

# Ejemplo de dataset de evaluación
def create_sample_dataset() -> pd.DataFrame:
    """
    Crea un dataset de ejemplo para evaluación
    """
    sample_data = [
        {
            'question': '¿Cuál es la duración máxima de la jornada laboral?',
            'expected_answer': 'La duración máxima de la jornada laboral es de 8 horas diarias.',
            'expected_articles': ['art_123', 'art_456'],
            'category': 'jornada_laboral',
            'difficulty': 'easy'
        },
        {
            'question': '¿Qué derechos tiene un trabajador en caso de despido?',
            'expected_answer': 'El trabajador tiene derecho a indemnización, preaviso y otros beneficios.',
            'expected_articles': ['art_789', 'art_101'],
            'category': 'despido',
            'difficulty': 'medium'
        },
        {
            'question': '¿Cómo se calcula la indemnización por despido?',
            'expected_answer': 'La indemnización se calcula según la antigüedad y el salario del trabajador.',
            'expected_articles': ['art_202', 'art_303'],
            'category': 'indemnizacion',
            'difficulty': 'hard'
        }
    ]
    return pd.DataFrame(sample_data)

# Cargar dataset (usar create_sample_dataset() si no tienes un dataset real)
try:
    # Intentar cargar dataset real
    dataset = load_evaluation_dataset('data/evaluation_dataset.json')
    print(f"✅ Dataset cargado: {len(dataset)} preguntas")
except FileNotFoundError:
    # Usar dataset de ejemplo
    dataset = create_sample_dataset()
    print(f"⚠️  Usando dataset de ejemplo: {len(dataset)} preguntas")

print(f"\nEstructura del dataset:")
print(dataset.head())
print(f"\nColumnas: {list(dataset.columns)}")


## 4. Conexión con Qdrant y Carga de Modelos


In [ ]:
# Conexión con Qdrant
def connect_to_qdrant() -> QdrantClient:
    """
    Establece conexión con Qdrant
    """
    try:
        client = QdrantClient(url=QDRANT_CONFIG['url'])
        # Verificar conexión
        collections = client.get_collections()
        print(f"✅ Conectado a Qdrant. Colecciones disponibles: {len(collections.collections)}")
        return client
    except Exception as e:
        print(f"❌ Error conectando a Qdrant: {e}")
        raise

# Cargar modelos de embedding
def load_embedding_models() -> Dict[str, SentenceTransformer]:
    """
    Carga todos los modelos de embedding configurados
    """
    models = {}
    print("🔄 Cargando modelos de embedding...")
    
    for name, config in EMBEDDING_MODELS.items():
        try:
            print(f"  - Cargando {name}...")
            model = SentenceTransformer(config['model_name'])
            models[name] = model
            print(f"    ✅ {name} cargado")
        except Exception as e:
            print(f"    ❌ Error cargando {name}: {e}")
    
    print(f"\n✅ {len(models)} modelos de embedding cargados")
    return models

# Cargar modelos de re-ranking
def load_reranking_models() -> Dict[str, CrossEncoder]:
    """
    Carga todos los modelos de re-ranking configurados
    """
    models = {}
    print("🔄 Cargando modelos de re-ranking...")
    
    for name, config in RERANKING_MODELS.items():
        try:
            print(f"  - Cargando {name}...")
            model = CrossEncoder(config['model_name'])
            models[name] = model
            print(f"    ✅ {name} cargado")
        except Exception as e:
            print(f"    ❌ Error cargando {name}: {e}")
    
    print(f"\n✅ {len(models)} modelos de re-ranking cargados")
    return models

# Ejecutar carga
qdrant_client = connect_to_qdrant()
embedding_models = load_embedding_models()
reranking_models = load_reranking_models()


# 1. Introducción

## Objetivo de la Notebook

Esta notebook tiene como objetivo realizar una **evaluación integral del pipeline RAG** (Retrieval-Augmented Generation) para el sistema de consultas sobre derecho laboral paraguayo. La evaluación se realiza en múltiples niveles:

### 🎯 Niveles de Evaluación

1. **Nivel de Retrieval (Recuperación)**
   - Evaluación de diferentes modelos de embeddings
   - Comparación de performance en Qdrant
   - Métricas objetivas: Recall@k, Precision@k, nDCG@k, MRR

2. **Nivel de Re-ranking**
   - Evaluación de modelos cross-encoder
   - Mejora en la relevancia de documentos recuperados
   - Comparación antes vs después del re-ranking

3. **Nivel de Generación (LLM)**
   - Evaluación de respuestas generadas
   - Métricas subjetivas via LLM-as-a-judge
   - Análisis de coherencia, relevancia, completitud

### 📊 Dataset de Evaluación

El dataset contiene:
- **Preguntas**: Consultas reales sobre derecho laboral paraguayo
- **Respuestas esperadas**: Ground truth para comparación
- **Artículos relevantes**: Documentos que deberían ser recuperados
- **Categorías**: Clasificación por tipo de consulta
- **Dificultad**: Nivel de complejidad de la pregunta

### 🔧 Tecnologías Utilizadas

- **Qdrant**: Base de datos vectorial para almacenamiento y búsqueda
- **Sentence Transformers**: Modelos de embeddings para representación de texto
- **Cross-encoders**: Modelos de re-ranking para mejorar relevancia
- **OpenAI/Transformers**: Modelos de generación y evaluación
- **Scikit-learn**: Cálculo de métricas de evaluación

### 💾 Sistema de Persistencia

La notebook incluye un sistema para guardar y cargar evaluaciones:
- **Guardado automático**: Cada evaluación se guarda en JSON
- **Comparación histórica**: Posibilidad de comparar diferentes configuraciones
- **Reproducibilidad**: Configuraciones guardadas para replicar experimentos


In [ ]:
# Sistema de Persistencia de Evaluaciones
class EvaluationManager:
    """
    Maneja el guardado y carga de evaluaciones para comparación histórica
    """
    
    def __init__(self, results_dir: str = "evaluation_results"):
        self.results_dir = Path(results_dir)
        self.results_dir.mkdir(exist_ok=True)
    
    def save_evaluation(self, 
                       evaluation_name: str,
                       config: Dict[str, Any],
                       results: Dict[str, Any],
                       metadata: Optional[Dict[str, Any]] = None) -> str:
        """
        Guarda una evaluación completa con configuración y resultados
        
        Args:
            evaluation_name: Nombre único para la evaluación
            config: Configuración de modelos y parámetros usados
            results: Resultados de la evaluación
            metadata: Metadatos adicionales (timestamp, descripción, etc.)
        
        Returns:
            Ruta del archivo guardado
        """
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"{evaluation_name}_{timestamp}.json"
        filepath = self.results_dir / filename
        
        evaluation_data = {
            "evaluation_name": evaluation_name,
            "timestamp": timestamp,
            "config": config,
            "results": results,
            "metadata": metadata or {}
        }
        
        with open(filepath, 'w', encoding='utf-8') as f:
            json.dump(evaluation_data, f, indent=2, ensure_ascii=False)
        
        print(f"✅ Evaluación guardada: {filepath}")
        return str(filepath)
    
    def load_evaluation(self, filepath: str) -> Dict[str, Any]:
        """
        Carga una evaluación guardada
        
        Args:
            filepath: Ruta del archivo de evaluación
        
        Returns:
            Datos de la evaluación
        """
        with open(filepath, 'r', encoding='utf-8') as f:
            return json.load(f)
    
    def list_evaluations(self) -> List[Dict[str, Any]]:
        """
        Lista todas las evaluaciones guardadas
        
        Returns:
            Lista de evaluaciones con metadatos
        """
        evaluations = []
        for filepath in self.results_dir.glob("*.json"):
            try:
                with open(filepath, 'r', encoding='utf-8') as f:
                    data = json.load(f)
                evaluations.append({
                    "filepath": str(filepath),
                    "name": data.get("evaluation_name", "unknown"),
                    "timestamp": data.get("timestamp", "unknown"),
                    "metadata": data.get("metadata", {})
                })
            except Exception as e:
                print(f"⚠️  Error cargando {filepath}: {e}")
        
        return sorted(evaluations, key=lambda x: x["timestamp"], reverse=True)
    
    def compare_evaluations(self, evaluation_paths: List[str]) -> pd.DataFrame:
        """
        Compara múltiples evaluaciones en una tabla
        
        Args:
            evaluation_paths: Lista de rutas de evaluaciones a comparar
        
        Returns:
            DataFrame con comparación de métricas
        """
        comparison_data = []
        
        for path in evaluation_paths:
            eval_data = self.load_evaluation(path)
            name = eval_data["evaluation_name"]
            timestamp = eval_data["timestamp"]
            results = eval_data["results"]
            
            # Extraer métricas principales
            row = {
                "evaluation": name,
                "timestamp": timestamp,
                "config": eval_data["config"]
            }
            
            # Agregar métricas de retrieval si existen
            if "retrieval_metrics" in results:
                for metric, values in results["retrieval_metrics"].items():
                    if isinstance(values, dict):
                        for k, v in values.items():
                            row[f"retrieval_{metric}_{k}"] = v
                    else:
                        row[f"retrieval_{metric}"] = values
            
            # Agregar métricas de LLM si existen
            if "llm_metrics" in results:
                for metric, values in results["llm_metrics"].items():
                    if isinstance(values, dict):
                        for k, v in values.items():
                            row[f"llm_{metric}_{k}"] = v
                    else:
                        row[f"llm_{metric}"] = values
            
            comparison_data.append(row)
        
        return pd.DataFrame(comparison_data)

# Inicializar el manager de evaluaciones
eval_manager = EvaluationManager()
print("✅ Sistema de persistencia de evaluaciones inicializado")


# 2. Carga del Dataset

## Estructura del Dataset de Evaluación

El dataset de evaluación debe contener preguntas y respuestas esperadas para poder medir la calidad del pipeline RAG. La estructura recomendada incluye:

### 📋 Campos Requeridos

- **`question`**: Pregunta a evaluar
- **`expected_answer`**: Respuesta esperada (ground truth)
- **`expected_articles`**: Lista de IDs de artículos relevantes que deberían ser recuperados
- **`category`**: Categoría de la pregunta (opcional)
- **`difficulty`**: Nivel de dificultad (opcional)

### 🎯 Tipos de Dataset Soportados

1. **Dataset Real**: Archivo JSON/CSV con preguntas reales del dominio
2. **Dataset de Ejemplo**: Dataset sintético para pruebas iniciales
3. **Dataset Híbrido**: Combinación de datos reales y sintéticos


In [ ]:
# Funciones de carga y validación del dataset
def validate_dataset(df: pd.DataFrame) -> Tuple[bool, List[str]]:
    """
    Valida que el dataset tenga la estructura correcta
    
    Args:
        df: DataFrame del dataset
    
    Returns:
        Tuple[bool, List[str]]: (es_válido, lista_de_errores)
    """
    errors = []
    required_columns = ['question', 'expected_answer', 'expected_articles']
    
    # Verificar columnas requeridas
    for col in required_columns:
        if col not in df.columns:
            errors.append(f"Columna requerida '{col}' no encontrada")
    
    if errors:
        return False, errors
    
    # Verificar que no haya valores nulos en columnas requeridas
    for col in required_columns:
        if df[col].isnull().any():
            errors.append(f"Columna '{col}' contiene valores nulos")
    
    # Verificar que expected_articles sea una lista
    if 'expected_articles' in df.columns:
        for idx, articles in df['expected_articles'].items():
            if not isinstance(articles, list):
                errors.append(f"Fila {idx}: expected_articles debe ser una lista")
    
    return len(errors) == 0, errors

def create_enhanced_sample_dataset() -> pd.DataFrame:
    """
    Crea un dataset de ejemplo más completo para evaluación
    """
    sample_data = [
        {
            'question': '¿Cuál es la duración máxima de la jornada laboral?',
            'expected_answer': 'La duración máxima de la jornada laboral es de 8 horas diarias según el artículo 154 del Código del Trabajo.',
            'expected_articles': ['art_154', 'art_155', 'art_156'],
            'category': 'jornada_laboral',
            'difficulty': 'easy',
            'keywords': ['jornada', 'horas', 'duración', 'máxima']
        },
        {
            'question': '¿Qué derechos tiene un trabajador en caso de despido?',
            'expected_answer': 'El trabajador tiene derecho a indemnización, preaviso, vacaciones proporcionales y otros beneficios según los artículos 91 y 92.',
            'expected_articles': ['art_91', 'art_92', 'art_93', 'art_94'],
            'category': 'despido',
            'difficulty': 'medium',
            'keywords': ['despido', 'derechos', 'indemnización', 'preaviso']
        },
        {
            'question': '¿Cómo se calcula la indemnización por despido?',
            'expected_answer': 'La indemnización se calcula multiplicando el salario diario por 30 días por cada año de antigüedad, con un mínimo de 15 días.',
            'expected_articles': ['art_91', 'art_92', 'art_95'],
            'category': 'indemnizacion',
            'difficulty': 'hard',
            'keywords': ['indemnización', 'cálculo', 'antigüedad', 'salario']
        },
        {
            'question': '¿Cuáles son las condiciones para el trabajo nocturno?',
            'expected_answer': 'El trabajo nocturno tiene condiciones especiales de horario, remuneración y descanso según los artículos 160-165.',
            'expected_articles': ['art_160', 'art_161', 'art_162', 'art_163', 'art_164', 'art_165'],
            'category': 'trabajo_nocturno',
            'difficulty': 'medium',
            'keywords': ['nocturno', 'horario', 'remuneración', 'descanso']
        },
        {
            'question': '¿Qué es el salario mínimo y cómo se establece?',
            'expected_answer': 'El salario mínimo es la remuneración mínima que debe recibir un trabajador, establecida por el Consejo Nacional del Salario Mínimo.',
            'expected_articles': ['art_200', 'art_201', 'art_202'],
            'category': 'salario',
            'difficulty': 'easy',
            'keywords': ['salario', 'mínimo', 'remuneración', 'consejo']
        }
    ]
    return pd.DataFrame(sample_data)

def load_and_prepare_dataset(file_path: Optional[str] = None) -> pd.DataFrame:
    """
    Carga y prepara el dataset de evaluación
    
    Args:
        file_path: Ruta del archivo de dataset (opcional)
    
    Returns:
        DataFrame preparado y validado
    """
    if file_path and Path(file_path).exists():
        print(f"🔄 Cargando dataset desde: {file_path}")
        df = load_evaluation_dataset(file_path)
    else:
        print("⚠️  Usando dataset de ejemplo")
        df = create_enhanced_sample_dataset()
    
    # Validar dataset
    is_valid, errors = validate_dataset(df)
    
    if not is_valid:
        print("❌ Errores en el dataset:")
        for error in errors:
            print(f"  - {error}")
        raise ValueError("Dataset no válido")
    
    print(f"✅ Dataset cargado y validado: {len(df)} preguntas")
    
    # Mostrar estadísticas del dataset
    print(f"\n📊 Estadísticas del dataset:")
    print(f"  - Total de preguntas: {len(df)}")
    if 'category' in df.columns:
        print(f"  - Categorías: {df['category'].nunique()}")
        print(f"  - Distribución por categoría:")
        for cat, count in df['category'].value_counts().items():
            print(f"    * {cat}: {count}")
    
    if 'difficulty' in df.columns:
        print(f"  - Distribución por dificultad:")
        for diff, count in df['difficulty'].value_counts().items():
            print(f"    * {diff}: {count}")
    
    return df

# Cargar el dataset
print("🔄 Cargando dataset de evaluación...")
dataset = load_and_prepare_dataset()


In [ ]:
# Visualización del dataset
def visualize_dataset_distribution(df: pd.DataFrame):
    """
    Visualiza la distribución del dataset
    """
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('Distribución del Dataset de Evaluación', fontsize=16, fontweight='bold')
    
    # Distribución por categoría
    if 'category' in df.columns:
        category_counts = df['category'].value_counts()
        axes[0, 0].pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%')
        axes[0, 0].set_title('Distribución por Categoría')
    
    # Distribución por dificultad
    if 'difficulty' in df.columns:
        difficulty_counts = df['difficulty'].value_counts()
        axes[0, 1].bar(difficulty_counts.index, difficulty_counts.values, color='skyblue')
        axes[0, 1].set_title('Distribución por Dificultad')
        axes[0, 1].set_xlabel('Dificultad')
        axes[0, 1].set_ylabel('Número de Preguntas')
    
    # Longitud de preguntas
    question_lengths = df['question'].str.len()
    axes[1, 0].hist(question_lengths, bins=20, color='lightgreen', alpha=0.7)
    axes[1, 0].set_title('Distribución de Longitud de Preguntas')
    axes[1, 0].set_xlabel('Caracteres')
    axes[1, 0].set_ylabel('Frecuencia')
    
    # Longitud de respuestas esperadas
    answer_lengths = df['expected_answer'].str.len()
    axes[1, 1].hist(answer_lengths, bins=20, color='lightcoral', alpha=0.7)
    axes[1, 1].set_title('Distribución de Longitud de Respuestas')
    axes[1, 1].set_xlabel('Caracteres')
    axes[1, 1].set_ylabel('Frecuencia')
    
    plt.tight_layout()
    plt.show()
    
    # Estadísticas adicionales
    print(f"\n📈 Estadísticas detalladas:")
    print(f"  - Longitud promedio de preguntas: {question_lengths.mean():.1f} caracteres")
    print(f"  - Longitud promedio de respuestas: {answer_lengths.mean():.1f} caracteres")
    print(f"  - Número promedio de artículos esperados: {df['expected_articles'].apply(len).mean():.1f}")

# Mostrar algunas preguntas de ejemplo
def show_sample_questions(df: pd.DataFrame, n: int = 3):
    """
    Muestra preguntas de ejemplo del dataset
    """
    print(f"\n📝 Ejemplos de preguntas del dataset:")
    print("=" * 80)
    
    for idx, row in df.head(n).iterrows():
        print(f"\n🔹 Pregunta {idx + 1}:")
        print(f"   Pregunta: {row['question']}")
        print(f"   Respuesta esperada: {row['expected_answer']}")
        print(f"   Artículos relevantes: {row['expected_articles']}")
        if 'category' in row:
            print(f"   Categoría: {row['category']}")
        if 'difficulty' in row:
            print(f"   Dificultad: {row['difficulty']}")
        print("-" * 80)

# Ejecutar visualizaciones
visualize_dataset_distribution(dataset)
show_sample_questions(dataset)


# 3. Configuración del Entorno

## Configuración de Servicios y Modelos

Esta sección establece la conexión con todos los servicios necesarios para la evaluación del pipeline RAG:

### 🔧 Servicios a Configurar

1. **Qdrant**: Base de datos vectorial para retrieval
2. **Modelos de Embedding**: Para representación de texto
3. **Modelos de Re-ranking**: Para mejorar relevancia de documentos
4. **LLM**: Para generación y evaluación de respuestas
5. **Sistema de Métricas**: Para cálculo de métricas de evaluación


In [ ]:
# Configuración avanzada del entorno
class EnvironmentManager:
    """
    Maneja la configuración y conexión de todos los servicios
    """
    
    def __init__(self):
        self.qdrant_client = None
        self.embedding_models = {}
        self.reranking_models = {}
        self.llm_pipeline = None
        self.evaluation_metrics = {}
        
    def setup_qdrant(self) -> bool:
        """
        Configura la conexión con Qdrant
        """
        try:
            print("🔄 Configurando Qdrant...")
            self.qdrant_client = QdrantClient(url=QDRANT_CONFIG['url'])
            
            # Verificar conexión
            collections = self.qdrant_client.get_collections()
            print(f"✅ Qdrant conectado. Colecciones: {len(collections.collections)}")
            
            # Verificar colección específica
            collection_name = QDRANT_CONFIG['collection_name']
            try:
                collection_info = self.qdrant_client.get_collection(collection_name)
                print(f"✅ Colección '{collection_name}' encontrada:")
                print(f"   - Puntos: {collection_info.points_count}")
                print(f"   - Dimensiones: {collection_info.config.params.vectors.size}")
                print(f"   - Distancia: {collection_info.config.params.vectors.distance}")
            except Exception as e:
                print(f"⚠️  Colección '{collection_name}' no encontrada: {e}")
                print("   Asegúrate de que la colección existe y tiene datos")
            
            return True
            
        except Exception as e:
            print(f"❌ Error configurando Qdrant: {e}")
            return False
    
    def setup_embedding_models(self) -> bool:
        """
        Carga todos los modelos de embedding configurados
        """
        try:
            print("🔄 Cargando modelos de embedding...")
            self.embedding_models = {}
            
            for name, config in EMBEDDING_MODELS.items():
                try:
                    print(f"  - Cargando {name}...")
                    model = SentenceTransformer(config['model_name'])
                    self.embedding_models[name] = {
                        'model': model,
                        'config': config
                    }
                    print(f"    ✅ {name} cargado (dimensión: {config['dimension']})")
                except Exception as e:
                    print(f"    ❌ Error cargando {name}: {e}")
            
            print(f"✅ {len(self.embedding_models)} modelos de embedding cargados")
            return len(self.embedding_models) > 0
            
        except Exception as e:
            print(f"❌ Error configurando modelos de embedding: {e}")
            return False
    
    def setup_reranking_models(self) -> bool:
        """
        Carga todos los modelos de re-ranking configurados
        """
        try:
            print("🔄 Cargando modelos de re-ranking...")
            self.reranking_models = {}
            
            for name, config in RERANKING_MODELS.items():
                try:
                    print(f"  - Cargando {name}...")
                    model = CrossEncoder(config['model_name'])
                    self.reranking_models[name] = {
                        'model': model,
                        'config': config
                    }
                    print(f"    ✅ {name} cargado")
                except Exception as e:
                    print(f"    ❌ Error cargando {name}: {e}")
            
            print(f"✅ {len(self.reranking_models)} modelos de re-ranking cargados")
            return len(self.reranking_models) > 0
            
        except Exception as e:
            print(f"❌ Error configurando modelos de re-ranking: {e}")
            return False
    
    def setup_llm_pipeline(self) -> bool:
        """
        Configura el pipeline de LLM para generación y evaluación
        """
        try:
            print("🔄 Configurando LLM pipeline...")
            
            if LLM_CONFIG['provider'] == 'openai':
                # Configurar OpenAI
                openai.api_key = os.getenv('OPENAI_API_KEY')
                if not openai.api_key:
                    print("⚠️  OPENAI_API_KEY no encontrada. Usando modelo local.")
                    return self._setup_local_llm()
                
                self.llm_pipeline = {
                    'provider': 'openai',
                    'model': LLM_CONFIG['model'],
                    'temperature': LLM_CONFIG['temperature'],
                    'max_tokens': LLM_CONFIG['max_tokens']
                }
                print(f"✅ OpenAI configurado: {LLM_CONFIG['model']}")
                
            else:
                return self._setup_local_llm()
            
            return True
            
        except Exception as e:
            print(f"❌ Error configurando LLM: {e}")
            return False
    
    def _setup_local_llm(self) -> bool:
        """
        Configura un modelo local como fallback
        """
        try:
            print("🔄 Configurando modelo local...")
            model_name = "microsoft/DialoGPT-medium"  # Modelo más ligero
            
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForCausalLM.from_pretrained(model_name)
            
            self.llm_pipeline = {
                'provider': 'huggingface',
                'model': model,
                'tokenizer': tokenizer,
                'model_name': model_name
            }
            
            print(f"✅ Modelo local configurado: {model_name}")
            return True
            
        except Exception as e:
            print(f"❌ Error configurando modelo local: {e}")
            return False
    
    def setup_evaluation_metrics(self) -> bool:
        """
        Configura el sistema de métricas de evaluación
        """
        try:
            print("🔄 Configurando sistema de métricas...")
            
            self.evaluation_metrics = {
                'retrieval_metrics': RETRIEVAL_METRICS,
                'llm_criteria': LLM_EVALUATION_CRITERIA,
                'custom_metrics': {}
            }
            
            print("✅ Sistema de métricas configurado")
            return True
            
        except Exception as e:
            print(f"❌ Error configurando métricas: {e}")
            return False
    
    def initialize_all(self) -> Dict[str, bool]:
        """
        Inicializa todos los servicios
        """
        print("🚀 Inicializando entorno completo...")
        print("=" * 50)
        
        results = {
            'qdrant': self.setup_qdrant(),
            'embedding_models': self.setup_embedding_models(),
            'reranking_models': self.setup_reranking_models(),
            'llm_pipeline': self.setup_llm_pipeline(),
            'evaluation_metrics': self.setup_evaluation_metrics()
        }
        
        print("\n" + "=" * 50)
        print("📊 Resumen de inicialización:")
        for service, status in results.items():
            status_icon = "✅" if status else "❌"
            print(f"  {status_icon} {service}: {'OK' if status else 'ERROR'}")
        
        all_ok = all(results.values())
        if all_ok:
            print("\n🎉 ¡Entorno completamente configurado!")
        else:
            print("\n⚠️  Algunos servicios no se configuraron correctamente")
        
        return results

# Inicializar el manager del entorno
env_manager = EnvironmentManager()
initialization_results = env_manager.initialize_all()


In [ ]:
# Funciones de utilidad para el entorno
def get_environment_status() -> Dict[str, Any]:
    """
    Obtiene el estado actual del entorno
    """
    status = {
        'qdrant': {
            'connected': env_manager.qdrant_client is not None,
            'collections_available': 0
        },
        'embedding_models': {
            'loaded': len(env_manager.embedding_models),
            'models': list(env_manager.embedding_models.keys())
        },
        'reranking_models': {
            'loaded': len(env_manager.reranking_models),
            'models': list(env_manager.reranking_models.keys())
        },
        'llm_pipeline': {
            'configured': env_manager.llm_pipeline is not None,
            'provider': env_manager.llm_pipeline.get('provider') if env_manager.llm_pipeline else None
        },
        'evaluation_metrics': {
            'configured': len(env_manager.evaluation_metrics) > 0
        }
    }
    
    # Obtener información adicional de Qdrant
    if env_manager.qdrant_client:
        try:
            collections = env_manager.qdrant_client.get_collections()
            status['qdrant']['collections_available'] = len(collections.collections)
        except:
            pass
    
    return status

def test_environment_connectivity() -> Dict[str, bool]:
    """
    Prueba la conectividad de todos los servicios
    """
    print("🧪 Probando conectividad del entorno...")
    print("=" * 40)
    
    tests = {}
    
    # Test Qdrant
    if env_manager.qdrant_client:
        try:
            collections = env_manager.qdrant_client.get_collections()
            tests['qdrant'] = True
            print("✅ Qdrant: Conectado")
        except Exception as e:
            tests['qdrant'] = False
            print(f"❌ Qdrant: Error - {e}")
    else:
        tests['qdrant'] = False
        print("❌ Qdrant: No configurado")
    
    # Test Embedding Models
    if env_manager.embedding_models:
        try:
            # Probar con un modelo
            test_model = list(env_manager.embedding_models.values())[0]['model']
            test_embedding = test_model.encode(["test"])
            tests['embedding_models'] = True
            print(f"✅ Embedding Models: {len(env_manager.embedding_models)} modelos funcionando")
        except Exception as e:
            tests['embedding_models'] = False
            print(f"❌ Embedding Models: Error - {e}")
    else:
        tests['embedding_models'] = False
        print("❌ Embedding Models: No configurados")
    
    # Test Re-ranking Models
    if env_manager.reranking_models:
        try:
            # Probar con un modelo
            test_model = list(env_manager.reranking_models.values())[0]['model']
            test_scores = test_model.predict([("test query", "test document")])
            tests['reranking_models'] = True
            print(f"✅ Re-ranking Models: {len(env_manager.reranking_models)} modelos funcionando")
        except Exception as e:
            tests['reranking_models'] = False
            print(f"❌ Re-ranking Models: Error - {e}")
    else:
        tests['reranking_models'] = False
        print("❌ Re-ranking Models: No configurados")
    
    # Test LLM Pipeline
    if env_manager.llm_pipeline:
        try:
            if env_manager.llm_pipeline['provider'] == 'openai':
                # Test básico de OpenAI
                tests['llm_pipeline'] = True
                print("✅ LLM Pipeline: OpenAI configurado")
            else:
                # Test modelo local
                tests['llm_pipeline'] = True
                print("✅ LLM Pipeline: Modelo local configurado")
        except Exception as e:
            tests['llm_pipeline'] = False
            print(f"❌ LLM Pipeline: Error - {e}")
    else:
        tests['llm_pipeline'] = False
        print("❌ LLM Pipeline: No configurado")
    
    print("=" * 40)
    successful_tests = sum(tests.values())
    total_tests = len(tests)
    print(f"📊 Resultado: {successful_tests}/{total_tests} servicios funcionando")
    
    return tests

def display_environment_info():
    """
    Muestra información detallada del entorno
    """
    print("\n🔍 Información del Entorno")
    print("=" * 50)
    
    status = get_environment_status()
    
    # Qdrant
    print(f"\n🗄️  Qdrant:")
    print(f"   - Conectado: {'Sí' if status['qdrant']['connected'] else 'No'}")
    print(f"   - Colecciones disponibles: {status['qdrant']['collections_available']}")
    
    # Embedding Models
    print(f"\n🧠 Modelos de Embedding:")
    print(f"   - Cargados: {status['embedding_models']['loaded']}")
    for model in status['embedding_models']['models']:
        print(f"     * {model}")
    
    # Re-ranking Models
    print(f"\n🔄 Modelos de Re-ranking:")
    print(f"   - Cargados: {status['reranking_models']['loaded']}")
    for model in status['reranking_models']['models']:
        print(f"     * {model}")
    
    # LLM Pipeline
    print(f"\n🤖 LLM Pipeline:")
    print(f"   - Configurado: {'Sí' if status['llm_pipeline']['configured'] else 'No'}")
    if status['llm_pipeline']['provider']:
        print(f"   - Proveedor: {status['llm_pipeline']['provider']}")
    
    # Evaluation Metrics
    print(f"\n📊 Métricas de Evaluación:")
    print(f"   - Configuradas: {'Sí' if status['evaluation_metrics']['configured'] else 'No'}")

# Ejecutar pruebas y mostrar información
connectivity_results = test_environment_connectivity()
display_environment_info()


# 4. Evaluación de Retrievers (Embeddings + Qdrant)

## Objetivo de la Evaluación

Esta sección evalúa la calidad del sistema de recuperación de documentos usando diferentes modelos de embeddings y Qdrant. Se miden métricas objetivas para determinar qué modelo de embedding funciona mejor para el dominio del derecho laboral paraguayo.

### 🎯 Métricas de Evaluación

- **Recall@k**: Proporción de documentos relevantes recuperados en los top-k
- **Precision@k**: Proporción de documentos relevantes entre los top-k recuperados
- **nDCG@k**: Normalized Discounted Cumulative Gain - considera el ranking
- **MRR**: Mean Reciprocal Rank - posición del primer documento relevante
- **Hit Rate@k**: Proporción de consultas que tienen al menos un documento relevante en top-k

### 📊 Proceso de Evaluación

1. **Generación de embeddings** para cada pregunta del dataset
2. **Búsqueda en Qdrant** usando diferentes modelos de embedding
3. **Cálculo de métricas** comparando con documentos esperados
4. **Análisis comparativo** entre modelos
5. **Visualización de resultados** para identificar patrones


In [ ]:
# Clase para evaluación de retrievers
class RetrieverEvaluator:
    """
    Evalúa la calidad de diferentes modelos de embedding para retrieval
    """
    
    def __init__(self, qdrant_client, embedding_models, collection_name, top_k=20):
        self.qdrant_client = qdrant_client
        self.embedding_models = embedding_models
        self.collection_name = collection_name
        self.top_k = top_k
        self.results = {}
    
    def search_documents(self, query: str, embedding_model_name: str, top_k: int = None) -> List[Dict[str, Any]]:
        """
        Busca documentos en Qdrant usando un modelo de embedding específico
        
        Args:
            query: Pregunta a buscar
            embedding_model_name: Nombre del modelo de embedding
            top_k: Número de documentos a recuperar
        
        Returns:
            Lista de documentos recuperados con scores
        """
        if top_k is None:
            top_k = self.top_k
        
        try:
            # Obtener modelo de embedding
            model_info = self.embedding_models[embedding_model_name]
            model = model_info['model']
            
            # Generar embedding de la consulta
            query_embedding = model.encode([query])[0].tolist()
            
            # Buscar en Qdrant
            search_results = self.qdrant_client.search(
                collection_name=self.collection_name,
                query_vector=query_embedding,
                limit=top_k,
                with_payload=True,
                with_vectors=False
            )
            
            # Formatear resultados
            documents = []
            for result in search_results:
                documents.append({
                    'id': result.id,
                    'score': result.score,
                    'payload': result.payload
                })
            
            return documents
            
        except Exception as e:
            print(f"❌ Error en búsqueda con {embedding_model_name}: {e}")
            return []
    
    def calculate_recall_at_k(self, retrieved_ids: List[str], expected_ids: List[str], k: int) -> float:
        """
        Calcula Recall@k
        
        Args:
            retrieved_ids: IDs de documentos recuperados
            expected_ids: IDs de documentos esperados
            k: Número de documentos a considerar
        
        Returns:
            Recall@k score
        """
        if not expected_ids:
            return 0.0
        
        retrieved_k = set(retrieved_ids[:k])
        expected_set = set(expected_ids)
        
        intersection = retrieved_k.intersection(expected_set)
        return len(intersection) / len(expected_set)
    
    def calculate_precision_at_k(self, retrieved_ids: List[str], expected_ids: List[str], k: int) -> float:
        """
        Calcula Precision@k
        
        Args:
            retrieved_ids: IDs de documentos recuperados
            expected_ids: IDs de documentos esperados
            k: Número de documentos a considerar
        
        Returns:
            Precision@k score
        """
        if k == 0:
            return 0.0
        
        retrieved_k = set(retrieved_ids[:k])
        expected_set = set(expected_ids)
        
        intersection = retrieved_k.intersection(expected_set)
        return len(intersection) / k
    
    def calculate_ndcg_at_k(self, retrieved_ids: List[str], expected_ids: List[str], k: int) -> float:
        """
        Calcula nDCG@k
        
        Args:
            retrieved_ids: IDs de documentos recuperados
            expected_ids: IDs de documentos esperados
            k: Número de documentos a considerar
        
        Returns:
            nDCG@k score
        """
        if not expected_ids or k == 0:
            return 0.0
        
        # Crear relevancia binaria
        relevance = [1 if doc_id in expected_ids else 0 for doc_id in retrieved_ids[:k]]
        
        # Calcular DCG
        dcg = 0.0
        for i, rel in enumerate(relevance):
            dcg += rel / np.log2(i + 2)  # i+2 porque el log2(1) = 0
        
        # Calcular IDCG (ideal DCG)
        ideal_relevance = [1] * min(len(expected_ids), k)
        idcg = 0.0
        for i, rel in enumerate(ideal_relevance):
            idcg += rel / np.log2(i + 2)
        
        return dcg / idcg if idcg > 0 else 0.0
    
    def calculate_mrr(self, retrieved_ids: List[str], expected_ids: List[str]) -> float:
        """
        Calcula Mean Reciprocal Rank
        
        Args:
            retrieved_ids: IDs de documentos recuperados
            expected_ids: IDs de documentos esperados
        
        Returns:
            MRR score
        """
        if not expected_ids:
            return 0.0
        
        expected_set = set(expected_ids)
        for i, doc_id in enumerate(retrieved_ids):
            if doc_id in expected_set:
                return 1.0 / (i + 1)
        
        return 0.0
    
    def calculate_hit_rate_at_k(self, retrieved_ids: List[str], expected_ids: List[str], k: int) -> float:
        """
        Calcula Hit Rate@k
        
        Args:
            retrieved_ids: IDs de documentos recuperados
            expected_ids: IDs de documentos esperados
            k: Número de documentos a considerar
        
        Returns:
            Hit Rate@k score (0 o 1)
        """
        if not expected_ids:
            return 0.0
        
        retrieved_k = set(retrieved_ids[:k])
        expected_set = set(expected_ids)
        
        intersection = retrieved_k.intersection(expected_set)
        return 1.0 if len(intersection) > 0 else 0.0
    
    def evaluate_single_query(self, query: str, expected_articles: List[str], 
                            embedding_model_name: str) -> Dict[str, Any]:
        """
        Evalúa una sola consulta con un modelo de embedding
        
        Args:
            query: Pregunta a evaluar
            expected_articles: Lista de artículos esperados
            embedding_model_name: Nombre del modelo de embedding
        
        Returns:
            Diccionario con métricas calculadas
        """
        # Buscar documentos
        documents = self.search_documents(query, embedding_model_name)
        retrieved_ids = [doc['id'] for doc in documents]
        
        # Calcular métricas
        metrics = {}
        
        # Recall@k
        for k in RETRIEVAL_METRICS['recall_at_k']:
            metrics[f'recall_at_{k}'] = self.calculate_recall_at_k(retrieved_ids, expected_articles, k)
        
        # Precision@k
        for k in RETRIEVAL_METRICS['precision_at_k']:
            metrics[f'precision_at_{k}'] = self.calculate_precision_at_k(retrieved_ids, expected_articles, k)
        
        # nDCG@k
        for k in RETRIEVAL_METRICS['ndcg_at_k']:
            metrics[f'ndcg_at_{k}'] = self.calculate_ndcg_at_k(retrieved_ids, expected_articles, k)
        
        # MRR
        if RETRIEVAL_METRICS['mrr']:
            metrics['mrr'] = self.calculate_mrr(retrieved_ids, expected_articles)
        
        # Hit Rate@k
        for k in RETRIEVAL_METRICS['hit_rate_at_k']:
            metrics[f'hit_rate_at_{k}'] = self.calculate_hit_rate_at_k(retrieved_ids, expected_articles, k)
        
        # Información adicional
        metrics['total_retrieved'] = len(retrieved_ids)
        metrics['total_expected'] = len(expected_articles)
        metrics['retrieved_ids'] = retrieved_ids[:10]  # Primeros 10 para debugging
        
        return metrics

# Inicializar evaluador
retriever_evaluator = RetrieverEvaluator(
    qdrant_client=env_manager.qdrant_client,
    embedding_models=env_manager.embedding_models,
    collection_name=QDRANT_CONFIG['collection_name'],
    top_k=QDRANT_CONFIG['top_k']
)

print("✅ Evaluador de retrievers inicializado")


In [ ]:
# Función para ejecutar evaluación completa de retrievers
def evaluate_all_retrievers(dataset: pd.DataFrame, embedding_models: Dict[str, Any]) -> Dict[str, Any]:
    """
    Evalúa todos los modelos de embedding en el dataset completo
    
    Args:
        dataset: DataFrame con preguntas y artículos esperados
        embedding_models: Diccionario con modelos de embedding
    
    Returns:
        Diccionario con resultados de evaluación
    """
    print("🚀 Iniciando evaluación completa de retrievers...")
    print(f"📊 Evaluando {len(dataset)} preguntas con {len(embedding_models)} modelos")
    print("=" * 60)
    
    all_results = {}
    
    for model_name in embedding_models.keys():
        print(f"\n🔄 Evaluando modelo: {model_name}")
        print("-" * 40)
        
        model_results = {
            'model_name': model_name,
            'query_results': [],
            'aggregated_metrics': {}
        }
        
        # Evaluar cada pregunta
        for idx, row in dataset.iterrows():
            query = row['question']
            expected_articles = row['expected_articles']
            
            print(f"  📝 Pregunta {idx + 1}/{len(dataset)}: {query[:50]}...")
            
            try:
                # Evaluar consulta
                query_metrics = retriever_evaluator.evaluate_single_query(
                    query=query,
                    expected_articles=expected_articles,
                    embedding_model_name=model_name
                )
                
                # Agregar información de la consulta
                query_metrics['query'] = query
                query_metrics['expected_articles'] = expected_articles
                query_metrics['query_id'] = idx
                
                model_results['query_results'].append(query_metrics)
                
                # Mostrar métricas principales
                print(f"    📈 Recall@5: {query_metrics['recall_at_5']:.3f}, "
                      f"Precision@5: {query_metrics['precision_at_5']:.3f}, "
                      f"nDCG@5: {query_metrics['ndcg_at_5']:.3f}")
                
            except Exception as e:
                print(f"    ❌ Error en pregunta {idx + 1}: {e}")
                continue
        
        # Calcular métricas agregadas
        if model_results['query_results']:
            print(f"\n  📊 Calculando métricas agregadas para {model_name}...")
            aggregated = calculate_aggregated_metrics(model_results['query_results'])
            model_results['aggregated_metrics'] = aggregated
            
            # Mostrar resumen
            print(f"    🎯 Resumen de {model_name}:")
            print(f"      - Recall@5 promedio: {aggregated['recall_at_5']['mean']:.3f}")
            print(f"      - Precision@5 promedio: {aggregated['precision_at_5']['mean']:.3f}")
            print(f"      - nDCG@5 promedio: {aggregated['ndcg_at_5']['mean']:.3f}")
            print(f"      - MRR promedio: {aggregated['mrr']['mean']:.3f}")
        
        all_results[model_name] = model_results
    
    print("\n" + "=" * 60)
    print("✅ Evaluación de retrievers completada")
    
    return all_results

def calculate_aggregated_metrics(query_results: List[Dict[str, Any]]) -> Dict[str, Dict[str, float]]:
    """
    Calcula métricas agregadas (promedio, desviación estándar) de los resultados de consultas
    
    Args:
        query_results: Lista de resultados de consultas individuales
    
    Returns:
        Diccionario con métricas agregadas
    """
    if not query_results:
        return {}
    
    # Obtener todas las métricas disponibles
    metric_names = [key for key in query_results[0].keys() 
                   if key not in ['query', 'expected_articles', 'query_id', 'retrieved_ids', 
                                 'total_retrieved', 'total_expected']]
    
    aggregated = {}
    
    for metric_name in metric_names:
        values = [result[metric_name] for result in query_results if metric_name in result]
        
        if values:
            aggregated[metric_name] = {
                'mean': np.mean(values),
                'std': np.std(values),
                'min': np.min(values),
                'max': np.max(values),
                'median': np.median(values)
            }
    
    return aggregated

# Ejecutar evaluación de retrievers
print("🔄 Iniciando evaluación de retrievers...")
retrieval_results = evaluate_all_retrievers(dataset, env_manager.embedding_models)


In [ ]:
# Visualización de resultados de retrievers
def visualize_retrieval_results(retrieval_results: Dict[str, Any]):
    """
    Visualiza los resultados de evaluación de retrievers
    """
    if not retrieval_results:
        print("❌ No hay resultados para visualizar")
        return
    
    # Preparar datos para visualización
    models = list(retrieval_results.keys())
    metrics_to_plot = ['recall_at_5', 'precision_at_5', 'ndcg_at_5', 'mrr']
    
    # Crear figura con subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle('Comparación de Modelos de Embedding - Métricas de Retrieval', 
                 fontsize=16, fontweight='bold')
    
    # Colores para cada modelo
    colors = plt.cm.Set3(np.linspace(0, 1, len(models)))
    
    for idx, metric in enumerate(metrics_to_plot):
        row = idx // 2
        col = idx % 2
        
        # Extraer datos para la métrica
        model_names = []
        mean_values = []
        std_values = []
        
        for model_name in models:
            if metric in retrieval_results[model_name]['aggregated_metrics']:
                model_names.append(model_name)
                mean_values.append(retrieval_results[model_name]['aggregated_metrics'][metric]['mean'])
                std_values.append(retrieval_results[model_name]['aggregated_metrics'][metric]['std'])
        
        if model_names:
            # Crear gráfico de barras con barras de error
            bars = axes[row, col].bar(model_names, mean_values, yerr=std_values, 
                                    color=colors[:len(model_names)], alpha=0.7, capsize=5)
            axes[row, col].set_title(f'{metric.replace("_", " ").title()}')
            axes[row, col].set_ylabel('Score')
            axes[row, col].set_ylim(0, 1)
            
            # Rotar etiquetas del eje x
            axes[row, col].tick_params(axis='x', rotation=45)
            
            # Agregar valores en las barras
            for bar, mean_val in zip(bars, mean_values):
                height = bar.get_height()
                axes[row, col].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                                  f'{mean_val:.3f}', ha='center', va='bottom', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    # Crear tabla comparativa
    print("\n📊 Tabla Comparativa de Modelos:")
    print("=" * 80)
    
    # Preparar datos para la tabla
    comparison_data = []
    for model_name in models:
        row = {'Modelo': model_name}
        aggregated = retrieval_results[model_name]['aggregated_metrics']
        
        for metric in metrics_to_plot:
            if metric in aggregated:
                row[metric] = f"{aggregated[metric]['mean']:.3f} ± {aggregated[metric]['std']:.3f}"
            else:
                row[metric] = "N/A"
        
        comparison_data.append(row)
    
    # Crear DataFrame y mostrar
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))
    
    # Identificar mejor modelo
    print("\n🏆 Análisis de Mejores Modelos:")
    print("-" * 40)
    
    for metric in metrics_to_plot:
        best_model = None
        best_score = -1
        
        for model_name in models:
            if metric in retrieval_results[model_name]['aggregated_metrics']:
                score = retrieval_results[model_name]['aggregated_metrics'][metric]['mean']
                if score > best_score:
                    best_score = score
                    best_model = model_name
        
        if best_model:
            print(f"  {metric.replace('_', ' ').title()}: {best_model} ({best_score:.3f})")

def analyze_retrieval_performance(retrieval_results: Dict[str, Any]):
    """
    Analiza el rendimiento de los retrievers en detalle
    """
    print("\n🔍 Análisis Detallado de Rendimiento:")
    print("=" * 50)
    
    for model_name, results in retrieval_results.items():
        print(f"\n📈 Modelo: {model_name}")
        print("-" * 30)
        
        aggregated = results['aggregated_metrics']
        query_results = results['query_results']
        
        if not aggregated:
            print("  ❌ No hay métricas disponibles")
            continue
        
        # Métricas principales
        print(f"  🎯 Métricas Principales:")
        for metric in ['recall_at_5', 'precision_at_5', 'ndcg_at_5', 'mrr']:
            if metric in aggregated:
                mean_val = aggregated[metric]['mean']
                std_val = aggregated[metric]['std']
                print(f"    {metric.replace('_', ' ').title()}: {mean_val:.3f} ± {std_val:.3f}")
        
        # Análisis de consultas
        if query_results:
            print(f"  📝 Análisis de Consultas:")
            
            # Mejores consultas (mayor recall@5)
            best_queries = sorted(query_results, key=lambda x: x.get('recall_at_5', 0), reverse=True)[:3]
            print(f"    Mejores consultas (Recall@5):")
            for i, query in enumerate(best_queries, 1):
                print(f"      {i}. {query['query'][:60]}... (Recall@5: {query['recall_at_5']:.3f})")
            
            # Peores consultas (menor recall@5)
            worst_queries = sorted(query_results, key=lambda x: x.get('recall_at_5', 0))[:3]
            print(f"    Peores consultas (Recall@5):")
            for i, query in enumerate(worst_queries, 1):
                print(f"      {i}. {query['query'][:60]}... (Recall@5: {query['recall_at_5']:.3f})")

# Ejecutar visualizaciones y análisis
visualize_retrieval_results(retrieval_results)
analyze_retrieval_performance(retrieval_results)


In [ ]:
# Guardar resultados de evaluación de retrievers
def save_retrieval_evaluation_results(retrieval_results: Dict[str, Any], 
                                    evaluation_name: str = "retrieval_evaluation") -> str:
    """
    Guarda los resultados de evaluación de retrievers
    
    Args:
        retrieval_results: Resultados de la evaluación
        evaluation_name: Nombre de la evaluación
    
    Returns:
        Ruta del archivo guardado
    """
    # Preparar configuración
    config = {
        'embedding_models': list(EMBEDDING_MODELS.keys()),
        'retrieval_metrics': RETRIEVAL_METRICS,
        'qdrant_config': QDRANT_CONFIG,
        'dataset_size': len(dataset)
    }
    
    # Preparar metadatos
    metadata = {
        'evaluation_type': 'retrieval',
        'description': 'Evaluación de modelos de embedding para retrieval en Qdrant',
        'timestamp': datetime.now().isoformat(),
        'total_queries': len(dataset),
        'total_models': len(retrieval_results)
    }
    
    # Guardar evaluación
    filepath = eval_manager.save_evaluation(
        evaluation_name=evaluation_name,
        config=config,
        results={'retrieval_metrics': retrieval_results},
        metadata=metadata
    )
    
    print(f"💾 Resultados de evaluación de retrievers guardados en: {filepath}")
    return filepath

# Guardar resultados
retrieval_filepath = save_retrieval_evaluation_results(retrieval_results)

# Resumen final de la evaluación de retrievers
print("\n" + "=" * 60)
print("📋 RESUMEN DE EVALUACIÓN DE RETRIEVERS")
print("=" * 60)

if retrieval_results:
    # Encontrar el mejor modelo general
    best_model = None
    best_overall_score = -1
    
    for model_name, results in retrieval_results.items():
        aggregated = results['aggregated_metrics']
        if aggregated:
            # Calcular score promedio de métricas principales
            main_metrics = ['recall_at_5', 'precision_at_5', 'ndcg_at_5', 'mrr']
            scores = [aggregated[metric]['mean'] for metric in main_metrics if metric in aggregated]
            if scores:
                avg_score = np.mean(scores)
                if avg_score > best_overall_score:
                    best_overall_score = avg_score
                    best_model = model_name
    
    if best_model:
        print(f"🏆 Mejor modelo general: {best_model}")
        print(f"   Score promedio: {best_overall_score:.3f}")
    
    # Estadísticas por modelo
    print(f"\n📊 Estadísticas por modelo:")
    for model_name, results in retrieval_results.items():
        aggregated = results['aggregated_metrics']
        if aggregated and 'recall_at_5' in aggregated:
            recall = aggregated['recall_at_5']['mean']
            precision = aggregated['precision_at_5']['mean']
            ndcg = aggregated['ndcg_at_5']['mean']
            mrr = aggregated['mrr']['mean']
            
            print(f"  {model_name}:")
            print(f"    - Recall@5: {recall:.3f}")
            print(f"    - Precision@5: {precision:.3f}")
            print(f"    - nDCG@5: {ndcg:.3f}")
            print(f"    - MRR: {mrr:.3f}")
    
    print(f"\n✅ Evaluación completada exitosamente")
    print(f"📁 Resultados guardados en: {retrieval_filepath}")
else:
    print("❌ No se pudieron obtener resultados de evaluación")


# 5. Evaluación con Re-ranking

## Objetivo de la Evaluación

Esta sección evalúa cómo los modelos de re-ranking (cross-encoders) mejoran la calidad de los documentos recuperados. Los cross-encoders consideran tanto la consulta como el documento de forma conjunta, lo que les permite hacer predicciones más precisas sobre la relevancia.

### 🎯 Proceso de Re-ranking

1. **Retrieval inicial**: Obtener documentos usando embeddings (bi-encoder)
2. **Re-ranking**: Reordenar documentos usando cross-encoder
3. **Evaluación**: Comparar métricas antes y después del re-ranking
4. **Análisis**: Identificar mejoras en relevancia y ranking

### 📊 Métricas de Evaluación

- **Mejora en Recall@k**: Incremento en recuperación de documentos relevantes
- **Mejora en nDCG@k**: Mejora en la calidad del ranking
- **Mejora en MRR**: Mejora en la posición del primer documento relevante
- **Análisis de ranking**: Cambios en el orden de documentos
- **Eficiencia**: Tiempo adicional vs. mejora en calidad

### 🔄 Modelos de Re-ranking Evaluados

- **ms-marco-MiniLM-L-6-v2**: Modelo ligero y rápido
- **ms-marco-MiniLM-L-12-v2**: Modelo más robusto
- **ms-marco-MiniLM-L-2-v2**: Modelo ultra-ligero


In [ ]:
# Clase para evaluación de re-ranking
class RerankingEvaluator:
    """
    Evalúa la mejora en calidad de documentos mediante re-ranking
    """
    
    def __init__(self, qdrant_client, embedding_models, reranking_models, collection_name, top_k=20):
        self.qdrant_client = qdrant_client
        self.embedding_models = embedding_models
        self.reranking_models = reranking_models
        self.collection_name = collection_name
        self.top_k = top_k
        self.results = {}
    
    def retrieve_documents(self, query: str, embedding_model_name: str, top_k: int = None) -> List[Dict[str, Any]]:
        """
        Recupera documentos usando embeddings (sin re-ranking)
        
        Args:
            query: Pregunta a buscar
            embedding_model_name: Nombre del modelo de embedding
            top_k: Número de documentos a recuperar
        
        Returns:
            Lista de documentos recuperados
        """
        if top_k is None:
            top_k = self.top_k
        
        try:
            # Obtener modelo de embedding
            model_info = self.embedding_models[embedding_model_name]
            model = model_info['model']
            
            # Generar embedding de la consulta
            query_embedding = model.encode([query])[0].tolist()
            
            # Buscar en Qdrant
            search_results = self.qdrant_client.search(
                collection_name=self.collection_name,
                query_vector=query_embedding,
                limit=top_k,
                with_payload=True,
                with_vectors=False
            )
            
            # Formatear resultados
            documents = []
            for result in search_results:
                documents.append({
                    'id': result.id,
                    'score': result.score,
                    'payload': result.payload
                })
            
            return documents
            
        except Exception as e:
            print(f"❌ Error en retrieval con {embedding_model_name}: {e}")
            return []
    
    def rerank_documents(self, query: str, documents: List[Dict[str, Any]], 
                        reranking_model_name: str) -> List[Dict[str, Any]]:
        """
        Re-ordena documentos usando un modelo de re-ranking
        
        Args:
            query: Pregunta original
            documents: Lista de documentos a re-ordenar
            reranking_model_name: Nombre del modelo de re-ranking
        
        Returns:
            Lista de documentos re-ordenados con nuevos scores
        """
        if not documents:
            return []
        
        try:
            # Obtener modelo de re-ranking
            model_info = self.reranking_models[reranking_model_name]
            model = model_info['model']
            
            # Preparar pares (query, documento) para el cross-encoder
            query_doc_pairs = []
            for doc in documents:
                # Extraer texto del documento para re-ranking
                doc_text = self._extract_document_text(doc['payload'])
                query_doc_pairs.append((query, doc_text))
            
            # Calcular scores de relevancia
            relevance_scores = model.predict(query_doc_pairs)
            
            # Crear documentos con nuevos scores
            reranked_docs = []
            for i, doc in enumerate(documents):
                reranked_doc = doc.copy()
                reranked_doc['rerank_score'] = float(relevance_scores[i])
                reranked_doc['original_score'] = doc['score']
                reranked_docs.append(reranked_doc)
            
            # Ordenar por score de re-ranking
            reranked_docs.sort(key=lambda x: x['rerank_score'], reverse=True)
            
            return reranked_docs
            
        except Exception as e:
            print(f"❌ Error en re-ranking con {reranking_model_name}: {e}")
            return documents  # Retornar documentos originales si hay error
    
    def _extract_document_text(self, payload: Dict[str, Any]) -> str:
        """
        Extrae el texto relevante del payload del documento para re-ranking
        
        Args:
            payload: Payload del documento de Qdrant
        
        Returns:
            Texto del documento para re-ranking
        """
        # Combinar campos relevantes para re-ranking
        text_parts = []
        
        # Agregar capítulo y artículo (mismo formato que para embedding)
        if 'capitulo_descripcion' in payload and 'articulo' in payload:
            text_parts.append(f"{payload['capitulo_descripcion']}: {payload['articulo']}")
        
        # Agregar otros campos relevantes si existen
        for field in ['titulo', 'libro', 'capitulo']:
            if field in payload and payload[field]:
                text_parts.append(str(payload[field]))
        
        return " | ".join(text_parts) if text_parts else str(payload.get('articulo', ''))
    
    def evaluate_reranking_improvement(self, query: str, expected_articles: List[str],
                                     embedding_model_name: str, reranking_model_name: str) -> Dict[str, Any]:
        """
        Evalúa la mejora del re-ranking para una consulta específica
        
        Args:
            query: Pregunta a evaluar
            expected_articles: Lista de artículos esperados
            embedding_model_name: Nombre del modelo de embedding
            reranking_model_name: Nombre del modelo de re-ranking
        
        Returns:
            Diccionario con métricas de mejora
        """
        # 1. Retrieval inicial (sin re-ranking)
        initial_docs = self.retrieve_documents(query, embedding_model_name)
        initial_ids = [doc['id'] for doc in initial_docs]
        
        # 2. Re-ranking
        reranked_docs = self.rerank_documents(query, initial_docs, reranking_model_name)
        reranked_ids = [doc['id'] for doc in reranked_docs]
        
        # 3. Calcular métricas antes y después
        metrics = {}
        
        # Métricas iniciales (sin re-ranking)
        for k in RETRIEVAL_METRICS['recall_at_k']:
            metrics[f'initial_recall_at_{k}'] = self._calculate_recall_at_k(initial_ids, expected_articles, k)
        
        for k in RETRIEVAL_METRICS['precision_at_k']:
            metrics[f'initial_precision_at_{k}'] = self._calculate_precision_at_k(initial_ids, expected_articles, k)
        
        for k in RETRIEVAL_METRICS['ndcg_at_k']:
            metrics[f'initial_ndcg_at_{k}'] = self._calculate_ndcg_at_k(initial_ids, expected_articles, k)
        
        if RETRIEVAL_METRICS['mrr']:
            metrics['initial_mrr'] = self._calculate_mrr(initial_ids, expected_articles)
        
        # Métricas después del re-ranking
        for k in RETRIEVAL_METRICS['recall_at_k']:
            metrics[f'reranked_recall_at_{k}'] = self._calculate_recall_at_k(reranked_ids, expected_articles, k)
        
        for k in RETRIEVAL_METRICS['precision_at_k']:
            metrics[f'reranked_precision_at_{k}'] = self._calculate_precision_at_k(reranked_ids, expected_articles, k)
        
        for k in RETRIEVAL_METRICS['ndcg_at_k']:
            metrics[f'reranked_ndcg_at_{k}'] = self._calculate_ndcg_at_k(reranked_ids, expected_articles, k)
        
        if RETRIEVAL_METRICS['mrr']:
            metrics['reranked_mrr'] = self._calculate_mrr(reranked_ids, expected_articles)
        
        # Calcular mejoras
        for k in RETRIEVAL_METRICS['recall_at_k']:
            initial_key = f'initial_recall_at_{k}'
            reranked_key = f'reranked_recall_at_{k}'
            metrics[f'recall_improvement_at_{k}'] = metrics[reranked_key] - metrics[initial_key]
        
        for k in RETRIEVAL_METRICS['precision_at_k']:
            initial_key = f'initial_precision_at_{k}'
            reranked_key = f'reranked_precision_at_{k}'
            metrics[f'precision_improvement_at_{k}'] = metrics[reranked_key] - metrics[initial_key]
        
        for k in RETRIEVAL_METRICS['ndcg_at_k']:
            initial_key = f'initial_ndcg_at_{k}'
            reranked_key = f'reranked_ndcg_at_{k}'
            metrics[f'ndcg_improvement_at_{k}'] = metrics[reranked_key] - metrics[initial_key]
        
        if RETRIEVAL_METRICS['mrr']:
            metrics['mrr_improvement'] = metrics['reranked_mrr'] - metrics['initial_mrr']
        
        # Información adicional
        metrics['total_documents'] = len(initial_docs)
        metrics['expected_articles'] = expected_articles
        metrics['initial_ranking'] = initial_ids[:10]
        metrics['reranked_ranking'] = reranked_ids[:10]
        
        return metrics
    
    def _calculate_recall_at_k(self, retrieved_ids: List[str], expected_ids: List[str], k: int) -> float:
        """Calcula Recall@k"""
        if not expected_ids:
            return 0.0
        retrieved_k = set(retrieved_ids[:k])
        expected_set = set(expected_ids)
        intersection = retrieved_k.intersection(expected_set)
        return len(intersection) / len(expected_set)
    
    def _calculate_precision_at_k(self, retrieved_ids: List[str], expected_ids: List[str], k: int) -> float:
        """Calcula Precision@k"""
        if k == 0:
            return 0.0
        retrieved_k = set(retrieved_ids[:k])
        expected_set = set(expected_ids)
        intersection = retrieved_k.intersection(expected_set)
        return len(intersection) / k
    
    def _calculate_ndcg_at_k(self, retrieved_ids: List[str], expected_ids: List[str], k: int) -> float:
        """Calcula nDCG@k"""
        if not expected_ids or k == 0:
            return 0.0
        relevance = [1 if doc_id in expected_ids else 0 for doc_id in retrieved_ids[:k]]
        dcg = 0.0
        for i, rel in enumerate(relevance):
            dcg += rel / np.log2(i + 2)
        ideal_relevance = [1] * min(len(expected_ids), k)
        idcg = 0.0
        for i, rel in enumerate(ideal_relevance):
            idcg += rel / np.log2(i + 2)
        return dcg / idcg if idcg > 0 else 0.0
    
    def _calculate_mrr(self, retrieved_ids: List[str], expected_ids: List[str]) -> float:
        """Calcula MRR"""
        if not expected_ids:
            return 0.0
        expected_set = set(expected_ids)
        for i, doc_id in enumerate(retrieved_ids):
            if doc_id in expected_set:
                return 1.0 / (i + 1)
        return 0.0

# Inicializar evaluador de re-ranking
reranking_evaluator = RerankingEvaluator(
    qdrant_client=env_manager.qdrant_client,
    embedding_models=env_manager.embedding_models,
    reranking_models=env_manager.reranking_models,
    collection_name=QDRANT_CONFIG['collection_name'],
    top_k=QDRANT_CONFIG['top_k']
)

print("✅ Evaluador de re-ranking inicializado")


In [ ]:
# Función para evaluación completa de re-ranking
def evaluate_all_reranking_combinations(dataset: pd.DataFrame, 
                                      embedding_models: Dict[str, Any],
                                      reranking_models: Dict[str, Any]) -> Dict[str, Any]:
    """
    Evalúa todas las combinaciones de modelos de embedding y re-ranking
    
    Args:
        dataset: DataFrame con preguntas y artículos esperados
        embedding_models: Diccionario con modelos de embedding
        reranking_models: Diccionario con modelos de re-ranking
    
    Returns:
        Diccionario con resultados de evaluación
    """
    print("🚀 Iniciando evaluación completa de re-ranking...")
    print(f"📊 Evaluando {len(dataset)} preguntas")
    print(f"🔄 Combinaciones: {len(embedding_models)} embeddings × {len(reranking_models)} re-rankers")
    print("=" * 70)
    
    all_results = {}
    
    # Evaluar cada combinación de embedding + re-ranking
    for embedding_name in embedding_models.keys():
        print(f"\n🧠 Embedding: {embedding_name}")
        print("-" * 50)
        
        embedding_results = {}
        
        for reranking_name in reranking_models.keys():
            print(f"\n  🔄 Re-ranking: {reranking_name}")
            print("  " + "-" * 30)
            
            combination_key = f"{embedding_name}+{reranking_name}"
            combination_results = {
                'embedding_model': embedding_name,
                'reranking_model': reranking_name,
                'query_results': [],
                'aggregated_metrics': {}
            }
            
            # Evaluar cada pregunta
            for idx, row in dataset.iterrows():
                query = row['question']
                expected_articles = row['expected_articles']
                
                print(f"    📝 Pregunta {idx + 1}/{len(dataset)}: {query[:40]}...")
                
                try:
                    # Evaluar mejora del re-ranking
                    query_metrics = reranking_evaluator.evaluate_reranking_improvement(
                        query=query,
                        expected_articles=expected_articles,
                        embedding_model_name=embedding_name,
                        reranking_model_name=reranking_name
                    )
                    
                    # Agregar información de la consulta
                    query_metrics['query'] = query
                    query_metrics['expected_articles'] = expected_articles
                    query_metrics['query_id'] = idx
                    
                    combination_results['query_results'].append(query_metrics)
                    
                    # Mostrar métricas principales de mejora
                    recall_improvement = query_metrics.get('recall_improvement_at_5', 0)
                    ndcg_improvement = query_metrics.get('ndcg_improvement_at_5', 0)
                    mrr_improvement = query_metrics.get('mrr_improvement', 0)
                    
                    print(f"      📈 Mejora Recall@5: {recall_improvement:+.3f}, "
                          f"nDCG@5: {ndcg_improvement:+.3f}, MRR: {mrr_improvement:+.3f}")
                    
                except Exception as e:
                    print(f"      ❌ Error en pregunta {idx + 1}: {e}")
                    continue
            
            # Calcular métricas agregadas
            if combination_results['query_results']:
                print(f"\n    📊 Calculando métricas agregadas...")
                aggregated = calculate_reranking_aggregated_metrics(combination_results['query_results'])
                combination_results['aggregated_metrics'] = aggregated
                
                # Mostrar resumen de mejoras
                print(f"    🎯 Resumen de mejoras:")
                print(f"      - Recall@5: {aggregated.get('recall_improvement_at_5', {}).get('mean', 0):+.3f}")
                print(f"      - Precision@5: {aggregated.get('precision_improvement_at_5', {}).get('mean', 0):+.3f}")
                print(f"      - nDCG@5: {aggregated.get('ndcg_improvement_at_5', {}).get('mean', 0):+.3f}")
                print(f"      - MRR: {aggregated.get('mrr_improvement', {}).get('mean', 0):+.3f}")
            
            embedding_results[reranking_name] = combination_results
        
        all_results[embedding_name] = embedding_results
    
    print("\n" + "=" * 70)
    print("✅ Evaluación de re-ranking completada")
    
    return all_results

def calculate_reranking_aggregated_metrics(query_results: List[Dict[str, Any]]) -> Dict[str, Dict[str, float]]:
    """
    Calcula métricas agregadas para evaluación de re-ranking
    
    Args:
        query_results: Lista de resultados de consultas individuales
    
    Returns:
        Diccionario con métricas agregadas
    """
    if not query_results:
        return {}
    
    # Obtener todas las métricas disponibles
    metric_names = [key for key in query_results[0].keys() 
                   if key not in ['query', 'expected_articles', 'query_id', 'initial_ranking', 
                                 'reranked_ranking', 'total_documents']]
    
    aggregated = {}
    
    for metric_name in metric_names:
        values = [result[metric_name] for result in query_results if metric_name in result]
        
        if values:
            aggregated[metric_name] = {
                'mean': np.mean(values),
                'std': np.std(values),
                'min': np.min(values),
                'max': np.max(values),
                'median': np.median(values)
            }
    
    return aggregated

# Ejecutar evaluación de re-ranking
print("🔄 Iniciando evaluación de re-ranking...")
reranking_results = evaluate_all_reranking_combinations(
    dataset, 
    env_manager.embedding_models, 
    env_manager.reranking_models
)


In [ ]:
# Visualización de resultados de re-ranking
def visualize_reranking_results(reranking_results: Dict[str, Any]):
    """
    Visualiza los resultados de evaluación de re-ranking
    """
    if not reranking_results:
        print("❌ No hay resultados para visualizar")
        return
    
    # Preparar datos para visualización
    combinations = []
    improvement_data = []
    
    for embedding_name, embedding_data in reranking_results.items():
        for reranking_name, combination_data in embedding_data.items():
            if combination_data['aggregated_metrics']:
                combination_key = f"{embedding_name}\n+ {reranking_name}"
                combinations.append(combination_key)
                
                # Extraer mejoras principales
                metrics = combination_data['aggregated_metrics']
                improvement_data.append({
                    'combination': combination_key,
                    'recall_improvement': metrics.get('recall_improvement_at_5', {}).get('mean', 0),
                    'precision_improvement': metrics.get('precision_improvement_at_5', {}).get('mean', 0),
                    'ndcg_improvement': metrics.get('ndcg_improvement_at_5', {}).get('mean', 0),
                    'mrr_improvement': metrics.get('mrr_improvement', {}).get('mean', 0)
                })
    
    if not improvement_data:
        print("❌ No hay datos de mejora para visualizar")
        return
    
    # Crear figura con subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Mejoras por Re-ranking - Comparación de Combinaciones', 
                 fontsize=16, fontweight='bold')
    
    metrics_to_plot = ['recall_improvement', 'precision_improvement', 'ndcg_improvement', 'mrr_improvement']
    metric_titles = ['Recall@5', 'Precision@5', 'nDCG@5', 'MRR']
    
    # Colores para cada combinación
    colors = plt.cm.Set3(np.linspace(0, 1, len(combinations)))
    
    for idx, (metric, title) in enumerate(zip(metrics_to_plot, metric_titles)):
        row = idx // 2
        col = idx % 2
        
        # Extraer datos para la métrica
        values = [data[metric] for data in improvement_data]
        combination_labels = [data['combination'] for data in improvement_data]
        
        # Crear gráfico de barras
        bars = axes[row, col].bar(range(len(values)), values, color=colors, alpha=0.7)
        axes[row, col].set_title(f'Mejora en {title}')
        axes[row, col].set_ylabel('Mejora (Δ)')
        axes[row, col].set_xticks(range(len(combination_labels)))
        axes[row, col].set_xticklabels(combination_labels, rotation=45, ha='right')
        
        # Línea de referencia en 0
        axes[row, col].axhline(y=0, color='black', linestyle='--', alpha=0.5)
        
        # Agregar valores en las barras
        for bar, value in zip(bars, values):
            height = bar.get_height()
            axes[row, col].text(bar.get_x() + bar.get_width()/2., height + (0.001 if height >= 0 else -0.003),
                              f'{value:+.3f}', ha='center', va='bottom' if height >= 0 else 'top', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    # Crear tabla comparativa de mejoras
    print("\n📊 Tabla Comparativa de Mejoras por Re-ranking:")
    print("=" * 100)
    
    comparison_df = pd.DataFrame(improvement_data)
    comparison_df = comparison_df.round(4)
    print(comparison_df.to_string(index=False))
    
    # Identificar mejores combinaciones
    print("\n🏆 Mejores Combinaciones por Métrica:")
    print("-" * 50)
    
    for metric, title in zip(metrics_to_plot, metric_titles):
        best_idx = comparison_df[metric].idxmax()
        best_combination = comparison_df.loc[best_idx, 'combination']
        best_value = comparison_df.loc[best_idx, metric]
        print(f"  {title}: {best_combination} ({best_value:+.3f})")

def analyze_reranking_performance(reranking_results: Dict[str, Any]):
    """
    Analiza el rendimiento del re-ranking en detalle
    """
    print("\n🔍 Análisis Detallado de Re-ranking:")
    print("=" * 60)
    
    for embedding_name, embedding_data in reranking_results.items():
        print(f"\n🧠 Embedding: {embedding_name}")
        print("-" * 40)
        
        for reranking_name, combination_data in embedding_data.items():
            print(f"\n  🔄 Re-ranking: {reranking_name}")
            print("  " + "-" * 30)
            
            aggregated = combination_data['aggregated_metrics']
            query_results = combination_data['query_results']
            
            if not aggregated:
                print("    ❌ No hay métricas disponibles")
                continue
            
            # Métricas de mejora principales
            print(f"  📈 Mejoras Promedio:")
            for metric in ['recall_improvement_at_5', 'precision_improvement_at_5', 
                          'ndcg_improvement_at_5', 'mrr_improvement']:
                if metric in aggregated:
                    mean_val = aggregated[metric]['mean']
                    std_val = aggregated[metric]['std']
                    print(f"    {metric.replace('_improvement', '').replace('_at_5', '@5').title()}: {mean_val:+.3f} ± {std_val:.3f}")
            
            # Análisis de consultas con mayor mejora
            if query_results:
                print(f"  📝 Consultas con Mayor Mejora (nDCG@5):")
                best_queries = sorted(query_results, 
                                    key=lambda x: x.get('ndcg_improvement_at_5', 0), 
                                    reverse=True)[:3]
                for i, query in enumerate(best_queries, 1):
                    improvement = query.get('ndcg_improvement_at_5', 0)
                    print(f"    {i}. {query['query'][:50]}... (Mejora: {improvement:+.3f})")
                
                # Análisis de consultas con peor rendimiento
                print(f"  📝 Consultas con Menor Mejora (nDCG@5):")
                worst_queries = sorted(query_results, 
                                     key=lambda x: x.get('ndcg_improvement_at_5', 0))[:3]
                for i, query in enumerate(worst_queries, 1):
                    improvement = query.get('ndcg_improvement_at_5', 0)
                    print(f"    {i}. {query['query'][:50]}... (Mejora: {improvement:+.3f})")

# Ejecutar visualizaciones y análisis
visualize_reranking_results(reranking_results)
analyze_reranking_performance(reranking_results)


In [ ]:
# Guardar resultados de evaluación de re-ranking
def save_reranking_evaluation_results(reranking_results: Dict[str, Any], 
                                    evaluation_name: str = "reranking_evaluation") -> str:
    """
    Guarda los resultados de evaluación de re-ranking
    
    Args:
        reranking_results: Resultados de la evaluación
        evaluation_name: Nombre de la evaluación
    
    Returns:
        Ruta del archivo guardado
    """
    # Preparar configuración
    config = {
        'embedding_models': list(EMBEDDING_MODELS.keys()),
        'reranking_models': list(RERANKING_MODELS.keys()),
        'retrieval_metrics': RETRIEVAL_METRICS,
        'qdrant_config': QDRANT_CONFIG,
        'dataset_size': len(dataset)
    }
    
    # Preparar metadatos
    metadata = {
        'evaluation_type': 'reranking',
        'description': 'Evaluación de modelos de re-ranking para mejorar relevancia de documentos',
        'timestamp': datetime.now().isoformat(),
        'total_queries': len(dataset),
        'total_combinations': sum(len(embedding_data) for embedding_data in reranking_results.values())
    }
    
    # Guardar evaluación
    filepath = eval_manager.save_evaluation(
        evaluation_name=evaluation_name,
        config=config,
        results={'reranking_metrics': reranking_results},
        metadata=metadata
    )
    
    print(f"💾 Resultados de evaluación de re-ranking guardados en: {filepath}")
    return filepath

# Guardar resultados
reranking_filepath = save_reranking_evaluation_results(reranking_results)

# Resumen final de la evaluación de re-ranking
print("\n" + "=" * 70)
print("📋 RESUMEN DE EVALUACIÓN DE RE-RANKING")
print("=" * 70)

if reranking_results:
    # Encontrar la mejor combinación general
    best_combination = None
    best_overall_improvement = -1
    
    for embedding_name, embedding_data in reranking_results.items():
        for reranking_name, combination_data in embedding_data.items():
            aggregated = combination_data['aggregated_metrics']
            if aggregated:
                # Calcular mejora promedio de métricas principales
                main_improvements = ['recall_improvement_at_5', 'precision_improvement_at_5', 
                                   'ndcg_improvement_at_5', 'mrr_improvement']
                improvements = [aggregated[metric]['mean'] for metric in main_improvements 
                              if metric in aggregated]
                if improvements:
                    avg_improvement = np.mean(improvements)
                    if avg_improvement > best_overall_improvement:
                        best_overall_improvement = avg_improvement
                        best_combination = f"{embedding_name} + {reranking_name}"
    
    if best_combination:
        print(f"🏆 Mejor combinación general: {best_combination}")
        print(f"   Mejora promedio: {best_overall_improvement:+.3f}")
    
    # Estadísticas por combinación
    print(f"\n📊 Estadísticas por combinación:")
    for embedding_name, embedding_data in reranking_results.items():
        print(f"\n  🧠 Embedding: {embedding_name}")
        for reranking_name, combination_data in embedding_data.items():
            aggregated = combination_data['aggregated_metrics']
            if aggregated and 'ndcg_improvement_at_5' in aggregated:
                recall_improvement = aggregated['recall_improvement_at_5']['mean']
                precision_improvement = aggregated['precision_improvement_at_5']['mean']
                ndcg_improvement = aggregated['ndcg_improvement_at_5']['mean']
                mrr_improvement = aggregated['mrr_improvement']['mean']
                
                print(f"    🔄 {reranking_name}:")
                print(f"      - Recall@5: {recall_improvement:+.3f}")
                print(f"      - Precision@5: {precision_improvement:+.3f}")
                print(f"      - nDCG@5: {ndcg_improvement:+.3f}")
                print(f"      - MRR: {mrr_improvement:+.3f}")
    
    # Análisis de efectividad del re-ranking
    print(f"\n📈 Análisis de Efectividad del Re-ranking:")
    print("-" * 50)
    
    total_combinations = 0
    positive_improvements = 0
    
    for embedding_data in reranking_results.values():
        for combination_data in embedding_data.values():
            aggregated = combination_data['aggregated_metrics']
            if aggregated and 'ndcg_improvement_at_5' in aggregated:
                total_combinations += 1
                if aggregated['ndcg_improvement_at_5']['mean'] > 0:
                    positive_improvements += 1
    
    if total_combinations > 0:
        effectiveness_rate = (positive_improvements / total_combinations) * 100
        print(f"  - Combinaciones que mejoran nDCG@5: {positive_improvements}/{total_combinations} ({effectiveness_rate:.1f}%)")
        
        if effectiveness_rate > 50:
            print("  ✅ El re-ranking es efectivo en la mayoría de combinaciones")
        elif effectiveness_rate > 25:
            print("  ⚠️  El re-ranking es moderadamente efectivo")
        else:
            print("  ❌ El re-ranking tiene efectividad limitada")
    
    print(f"\n✅ Evaluación de re-ranking completada exitosamente")
    print(f"📁 Resultados guardados en: {reranking_filepath}")
else:
    print("❌ No se pudieron obtener resultados de evaluación de re-ranking")


# 6. Evaluación del Flujo Completo con LLM

## Objetivo de la Evaluación

Esta sección evalúa el pipeline completo de RAG, incluyendo la generación de respuestas con LLM y la evaluación de calidad subjetiva usando LLM-as-a-judge. Se combinan métricas objetivas de retrieval con métricas subjetivas de calidad de respuesta.

### 🎯 Proceso de Evaluación Completa

1. **Retrieval**: Obtener documentos relevantes usando embeddings + Qdrant
2. **Re-ranking**: Mejorar relevancia con cross-encoders (opcional)
3. **Generación**: Crear respuesta usando LLM con contexto recuperado
4. **Evaluación**: Medir calidad subjetiva con LLM-as-a-judge
5. **Análisis**: Combinar métricas objetivas y subjetivas

### 📊 Métricas de Evaluación

#### Métricas Objetivas (Retrieval)
- **Recall@k**: Proporción de documentos relevantes recuperados
- **Precision@k**: Proporción de documentos relevantes entre los recuperados
- **nDCG@k**: Calidad del ranking de documentos
- **MRR**: Posición del primer documento relevante

#### Métricas Subjetivas (LLM-as-a-judge)
- **Coherencia**: ¿La respuesta es coherente y bien estructurada?
- **Relevancia**: ¿La respuesta es relevante a la pregunta?
- **Completitud**: ¿La respuesta abarca todos los aspectos necesarios?
- **Fidelidad**: ¿La respuesta es fiel al contexto proporcionado?
- **Concisión**: ¿La respuesta es concisa sin ser incompleta?

### 🤖 Modelos de LLM Evaluados

- **OpenAI GPT-3.5-turbo**: Modelo comercial robusto
- **Modelo Local**: Fallback para entornos sin API externa
- **Configuración Flexible**: Temperatura, max_tokens, etc.


In [ ]:
# Clase para evaluación completa del pipeline RAG
class RAGPipelineEvaluator:
    """
    Evalúa el pipeline completo de RAG incluyendo generación y evaluación con LLM
    """
    
    def __init__(self, qdrant_client, embedding_models, reranking_models, 
                 llm_pipeline, collection_name, top_k=20):
        self.qdrant_client = qdrant_client
        self.embedding_models = embedding_models
        self.reranking_models = reranking_models
        self.llm_pipeline = llm_pipeline
        self.collection_name = collection_name
        self.top_k = top_k
        self.results = {}
    
    def retrieve_documents(self, query: str, embedding_model_name: str, 
                          top_k: int = None) -> List[Dict[str, Any]]:
        """
        Recupera documentos usando embeddings
        
        Args:
            query: Pregunta a buscar
            embedding_model_name: Nombre del modelo de embedding
            top_k: Número de documentos a recuperar
        
        Returns:
            Lista de documentos recuperados
        """
        if top_k is None:
            top_k = self.top_k
        
        try:
            # Obtener modelo de embedding
            model_info = self.embedding_models[embedding_model_name]
            model = model_info['model']
            
            # Generar embedding de la consulta
            query_embedding = model.encode([query])[0].tolist()
            
            # Buscar en Qdrant
            search_results = self.qdrant_client.search(
                collection_name=self.collection_name,
                query_vector=query_embedding,
                limit=top_k,
                with_payload=True,
                with_vectors=False
            )
            
            # Formatear resultados
            documents = []
            for result in search_results:
                documents.append({
                    'id': result.id,
                    'score': result.score,
                    'payload': result.payload
                })
            
            return documents
            
        except Exception as e:
            print(f"❌ Error en retrieval con {embedding_model_name}: {e}")
            return []
    
    def rerank_documents(self, query: str, documents: List[Dict[str, Any]], 
                        reranking_model_name: str) -> List[Dict[str, Any]]:
        """
        Re-ordena documentos usando re-ranking
        
        Args:
            query: Pregunta original
            documents: Lista de documentos a re-ordenar
            reranking_model_name: Nombre del modelo de re-ranking
        
        Returns:
            Lista de documentos re-ordenados
        """
        if not documents or not reranking_model_name:
            return documents
        
        try:
            # Obtener modelo de re-ranking
            model_info = self.reranking_models[reranking_model_name]
            model = model_info['model']
            
            # Preparar pares (query, documento) para el cross-encoder
            query_doc_pairs = []
            for doc in documents:
                doc_text = self._extract_document_text(doc['payload'])
                query_doc_pairs.append((query, doc_text))
            
            # Calcular scores de relevancia
            relevance_scores = model.predict(query_doc_pairs)
            
            # Crear documentos con nuevos scores
            reranked_docs = []
            for i, doc in enumerate(documents):
                reranked_doc = doc.copy()
                reranked_doc['rerank_score'] = float(relevance_scores[i])
                reranked_doc['original_score'] = doc['score']
                reranked_docs.append(reranked_doc)
            
            # Ordenar por score de re-ranking
            reranked_docs.sort(key=lambda x: x['rerank_score'], reverse=True)
            
            return reranked_docs
            
        except Exception as e:
            print(f"❌ Error en re-ranking con {reranking_model_name}: {e}")
            return documents
    
    def _extract_document_text(self, payload: Dict[str, Any]) -> str:
        """
        Extrae el texto relevante del payload del documento
        """
        text_parts = []
        
        # Agregar capítulo y artículo
        if 'capitulo_descripcion' in payload and 'articulo' in payload:
            text_parts.append(f"{payload['capitulo_descripcion']}: {payload['articulo']}")
        
        # Agregar otros campos relevantes
        for field in ['titulo', 'libro', 'capitulo']:
            if field in payload and payload[field]:
                text_parts.append(str(payload[field]))
        
        return " | ".join(text_parts) if text_parts else str(payload.get('articulo', ''))
    
    def generate_response(self, query: str, documents: List[Dict[str, Any]], 
                         max_context_docs: int = 5) -> str:
        """
        Genera respuesta usando LLM con contexto de documentos
        
        Args:
            query: Pregunta original
            documents: Lista de documentos recuperados
            max_context_docs: Número máximo de documentos a usar como contexto
        
        Returns:
            Respuesta generada por el LLM
        """
        if not documents:
            return "No se encontraron documentos relevantes para responder la pregunta."
        
        try:
            # Preparar contexto con documentos más relevantes
            context_docs = documents[:max_context_docs]
            context_text = self._build_context(context_docs)
            
            # Crear prompt para el LLM
            prompt = self._create_prompt(query, context_text)
            
            # Generar respuesta
            if self.llm_pipeline['provider'] == 'openai':
                response = self._generate_openai_response(prompt)
            else:
                response = self._generate_local_response(prompt)
            
            return response
            
        except Exception as e:
            print(f"❌ Error generando respuesta: {e}")
            return f"Error generando respuesta: {str(e)}"
    
    def _build_context(self, documents: List[Dict[str, Any]]) -> str:
        """
        Construye el contexto a partir de los documentos
        """
        context_parts = []
        
        for i, doc in enumerate(documents, 1):
            payload = doc['payload']
            
            # Extraer información del documento
            articulo_numero = payload.get('articulo_numero', 'N/A')
            capitulo_descripcion = payload.get('capitulo_descripcion', '')
            articulo_text = payload.get('articulo', '')
            
            # Formatear documento
            doc_text = f"Artículo {articulo_numero}: {capitulo_descripcion}\n{articulo_text}"
            context_parts.append(f"Documento {i}:\n{doc_text}\n")
        
        return "\n".join(context_parts)
    
    def _create_prompt(self, query: str, context: str) -> str:
        """
        Crea el prompt para el LLM
        """
        prompt = f"""Eres un asistente especializado en derecho laboral paraguayo. Responde la pregunta del usuario basándote únicamente en el contexto proporcionado.

CONTEXTO:
{context}

PREGUNTA: {query}

INSTRUCCIONES:
- Responde de manera clara y precisa
- Basa tu respuesta únicamente en el contexto proporcionado
- Si el contexto no contiene información suficiente, indícalo claramente
- Cita los artículos específicos cuando sea relevante
- Mantén un tono profesional y técnico apropiado para el ámbito legal

RESPUESTA:"""
        
        return prompt
    
    def _generate_openai_response(self, prompt: str) -> str:
        """
        Genera respuesta usando OpenAI
        """
        try:
            response = openai.ChatCompletion.create(
                model=self.llm_pipeline['model'],
                messages=[
                    {"role": "system", "content": "Eres un asistente especializado en derecho laboral paraguayo."},
                    {"role": "user", "content": prompt}
                ],
                temperature=self.llm_pipeline['temperature'],
                max_tokens=self.llm_pipeline['max_tokens']
            )
            
            return response.choices[0].message.content.strip()
            
        except Exception as e:
            print(f"❌ Error con OpenAI: {e}")
            return f"Error con OpenAI: {str(e)}"
    
    def _generate_local_response(self, prompt: str) -> str:
        """
        Genera respuesta usando modelo local
        """
        try:
            tokenizer = self.llm_pipeline['tokenizer']
            model = self.llm_pipeline['model']
            
            # Tokenizar input
            inputs = tokenizer.encode(prompt, return_tensors="pt", truncation=True, max_length=512)
            
            # Generar respuesta
            with torch.no_grad():
                outputs = model.generate(
                    inputs,
                    max_length=inputs.shape[1] + self.llm_pipeline.get('max_tokens', 200),
                    temperature=self.llm_pipeline.get('temperature', 0.7),
                    do_sample=True,
                    pad_token_id=tokenizer.eos_token_id
                )
            
            # Decodificar respuesta
            response = tokenizer.decode(outputs[0], skip_special_tokens=True)
            
            # Extraer solo la parte de la respuesta (después del prompt)
            if "RESPUESTA:" in response:
                response = response.split("RESPUESTA:")[-1].strip()
            
            return response
            
        except Exception as e:
            print(f"❌ Error con modelo local: {e}")
            return f"Error con modelo local: {str(e)}"
    
    def evaluate_response_quality(self, query: str, generated_response: str, 
                                 expected_answer: str, context_documents: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Evalúa la calidad de la respuesta generada usando LLM-as-a-judge
        
        Args:
            query: Pregunta original
            generated_response: Respuesta generada por el LLM
            expected_answer: Respuesta esperada (ground truth)
            context_documents: Documentos usados como contexto
        
        Returns:
            Diccionario con métricas de calidad
        """
        try:
            # Crear prompt para evaluación
            evaluation_prompt = self._create_evaluation_prompt(
                query, generated_response, expected_answer, context_documents
            )
            
            # Generar evaluación
            if self.llm_pipeline['provider'] == 'openai':
                evaluation_response = self._generate_openai_response(evaluation_prompt)
            else:
                evaluation_response = self._generate_local_response(evaluation_prompt)
            
            # Parsear evaluación
            quality_scores = self._parse_evaluation_response(evaluation_response)
            
            return quality_scores
            
        except Exception as e:
            print(f"❌ Error evaluando calidad: {e}")
            return self._get_default_quality_scores()
    
    def _create_evaluation_prompt(self, query: str, generated_response: str, 
                                 expected_answer: str, context_documents: List[Dict[str, Any]]) -> str:
        """
        Crea el prompt para evaluación con LLM-as-a-judge
        """
        context_text = self._build_context(context_documents)
        
        prompt = f"""Eres un evaluador experto en sistemas de RAG. Evalúa la calidad de la respuesta generada según los criterios especificados.

PREGUNTA: {query}

RESPUESTA ESPERADA: {expected_answer}

RESPUESTA GENERADA: {generated_response}

CONTEXTO USADO: {context_text}

CRITERIOS DE EVALUACIÓN:
1. Coherencia (1-5): ¿La respuesta es coherente y bien estructurada?
2. Relevancia (1-5): ¿La respuesta es relevante a la pregunta?
3. Completitud (1-5): ¿La respuesta abarca todos los aspectos necesarios?
4. Fidelidad (1-5): ¿La respuesta es fiel al contexto proporcionado?
5. Concisión (1-5): ¿La respuesta es concisa sin ser incompleta?

EVALÚA CADA CRITERIO Y RESPONDE EN EL SIGUIENTE FORMATO JSON:
{{
    "coherence": X,
    "relevance": X,
    "completeness": X,
    "fidelity": X,
    "conciseness": X,
    "overall_score": X,
    "explanation": "Breve explicación de la evaluación"
}}

Donde X es un número del 1 al 5 para cada criterio."""
        
        return prompt
    
    def _parse_evaluation_response(self, evaluation_response: str) -> Dict[str, Any]:
        """
        Parsea la respuesta de evaluación del LLM
        """
        try:
            # Buscar JSON en la respuesta
            import re
            json_match = re.search(r'\{.*\}', evaluation_response, re.DOTALL)
            
            if json_match:
                json_str = json_match.group()
                evaluation_data = json.loads(json_str)
                
                # Validar que tenga las claves necesarias
                required_keys = ['coherence', 'relevance', 'completeness', 'fidelity', 'conciseness']
                if all(key in evaluation_data for key in required_keys):
                    return evaluation_data
            
            # Si no se puede parsear, usar valores por defecto
            return self._get_default_quality_scores()
            
        except Exception as e:
            print(f"❌ Error parseando evaluación: {e}")
            return self._get_default_quality_scores()
    
    def _get_default_quality_scores(self) -> Dict[str, Any]:
        """
        Retorna scores por defecto cuando hay error en la evaluación
        """
        return {
            "coherence": 3.0,
            "relevance": 3.0,
            "completeness": 3.0,
            "fidelity": 3.0,
            "conciseness": 3.0,
            "overall_score": 3.0,
            "explanation": "Error en evaluación automática"
        }
    
    def evaluate_complete_pipeline(self, query: str, expected_answer: str, 
                                  expected_articles: List[str], embedding_model_name: str,
                                  reranking_model_name: str = None) -> Dict[str, Any]:
        """
        Evalúa el pipeline completo de RAG
        
        Args:
            query: Pregunta a evaluar
            expected_answer: Respuesta esperada
            expected_articles: Lista de artículos esperados
            embedding_model_name: Nombre del modelo de embedding
            reranking_model_name: Nombre del modelo de re-ranking (opcional)
        
        Returns:
            Diccionario con métricas completas
        """
        results = {
            'query': query,
            'expected_answer': expected_answer,
            'expected_articles': expected_articles,
            'embedding_model': embedding_model_name,
            'reranking_model': reranking_model_name
        }
        
        # 1. Retrieval
        documents = self.retrieve_documents(query, embedding_model_name)
        results['retrieved_documents'] = len(documents)
        results['retrieved_ids'] = [doc['id'] for doc in documents]
        
        # 2. Re-ranking (opcional)
        if reranking_model_name and reranking_model_name in self.reranking_models:
            documents = self.rerank_documents(query, documents, reranking_model_name)
            results['reranked'] = True
        else:
            results['reranked'] = False
        
        # 3. Generación de respuesta
        generated_response = self.generate_response(query, documents)
        results['generated_response'] = generated_response
        
        # 4. Métricas objetivas de retrieval
        retrieved_ids = [doc['id'] for doc in documents]
        results['retrieval_metrics'] = self._calculate_retrieval_metrics(retrieved_ids, expected_articles)
        
        # 5. Evaluación de calidad de respuesta
        quality_scores = self.evaluate_response_quality(
            query, generated_response, expected_answer, documents
        )
        results['quality_scores'] = quality_scores
        
        return results
    
    def _calculate_retrieval_metrics(self, retrieved_ids: List[str], expected_ids: List[str]) -> Dict[str, float]:
        """
        Calcula métricas objetivas de retrieval
        """
        metrics = {}
        
        # Recall@k
        for k in RETRIEVAL_METRICS['recall_at_k']:
            metrics[f'recall_at_{k}'] = self._calculate_recall_at_k(retrieved_ids, expected_ids, k)
        
        # Precision@k
        for k in RETRIEVAL_METRICS['precision_at_k']:
            metrics[f'precision_at_{k}'] = self._calculate_precision_at_k(retrieved_ids, expected_ids, k)
        
        # nDCG@k
        for k in RETRIEVAL_METRICS['ndcg_at_k']:
            metrics[f'ndcg_at_{k}'] = self._calculate_ndcg_at_k(retrieved_ids, expected_ids, k)
        
        # MRR
        if RETRIEVAL_METRICS['mrr']:
            metrics['mrr'] = self._calculate_mrr(retrieved_ids, expected_ids)
        
        return metrics
    
    def _calculate_recall_at_k(self, retrieved_ids: List[str], expected_ids: List[str], k: int) -> float:
        """Calcula Recall@k"""
        if not expected_ids:
            return 0.0
        retrieved_k = set(retrieved_ids[:k])
        expected_set = set(expected_ids)
        intersection = retrieved_k.intersection(expected_set)
        return len(intersection) / len(expected_set)
    
    def _calculate_precision_at_k(self, retrieved_ids: List[str], expected_ids: List[str], k: int) -> float:
        """Calcula Precision@k"""
        if k == 0:
            return 0.0
        retrieved_k = set(retrieved_ids[:k])
        expected_set = set(expected_ids)
        intersection = retrieved_k.intersection(expected_set)
        return len(intersection) / k
    
    def _calculate_ndcg_at_k(self, retrieved_ids: List[str], expected_ids: List[str], k: int) -> float:
        """Calcula nDCG@k"""
        if not expected_ids or k == 0:
            return 0.0
        relevance = [1 if doc_id in expected_ids else 0 for doc_id in retrieved_ids[:k]]
        dcg = 0.0
        for i, rel in enumerate(relevance):
            dcg += rel / np.log2(i + 2)
        ideal_relevance = [1] * min(len(expected_ids), k)
        idcg = 0.0
        for i, rel in enumerate(ideal_relevance):
            idcg += rel / np.log2(i + 2)
        return dcg / idcg if idcg > 0 else 0.0
    
    def _calculate_mrr(self, retrieved_ids: List[str], expected_ids: List[str]) -> float:
        """Calcula MRR"""
        if not expected_ids:
            return 0.0
        expected_set = set(expected_ids)
        for i, doc_id in enumerate(retrieved_ids):
            if doc_id in expected_set:
                return 1.0 / (i + 1)
        return 0.0

# Inicializar evaluador del pipeline completo
rag_evaluator = RAGPipelineEvaluator(
    qdrant_client=env_manager.qdrant_client,
    embedding_models=env_manager.embedding_models,
    reranking_models=env_manager.reranking_models,
    llm_pipeline=env_manager.llm_pipeline,
    collection_name=QDRANT_CONFIG['collection_name'],
    top_k=QDRANT_CONFIG['top_k']
)

print("✅ Evaluador del pipeline RAG completo inicializado")


In [ ]:
# Función para evaluación completa del pipeline RAG
def evaluate_complete_rag_pipeline(dataset: pd.DataFrame, 
                                 embedding_models: Dict[str, Any],
                                 reranking_models: Dict[str, Any],
                                 use_reranking: bool = True) -> Dict[str, Any]:
    """
    Evalúa el pipeline completo de RAG con todas las combinaciones
    
    Args:
        dataset: DataFrame con preguntas y respuestas esperadas
        embedding_models: Diccionario con modelos de embedding
        reranking_models: Diccionario con modelos de re-ranking
        use_reranking: Si usar re-ranking o no
    
    Returns:
        Diccionario con resultados de evaluación
    """
    print("🚀 Iniciando evaluación completa del pipeline RAG...")
    print(f"📊 Evaluando {len(dataset)} preguntas")
    print(f"🧠 Modelos de embedding: {len(embedding_models)}")
    print(f"🔄 Modelos de re-ranking: {len(reranking_models) if use_reranking else 0}")
    print("=" * 70)
    
    all_results = {}
    
    # Evaluar cada combinación de embedding + re-ranking
    for embedding_name in embedding_models.keys():
        print(f"\n🧠 Embedding: {embedding_name}")
        print("-" * 50)
        
        embedding_results = {}
        
        # Si no usar re-ranking, evaluar solo con embedding
        if not use_reranking:
            print(f"\n  📝 Evaluando sin re-ranking...")
            print("  " + "-" * 30)
            
            combination_results = {
                'embedding_model': embedding_name,
                'reranking_model': None,
                'query_results': [],
                'aggregated_metrics': {}
            }
            
            # Evaluar cada pregunta
            for idx, row in dataset.iterrows():
                query = row['question']
                expected_answer = row['expected_answer']
                expected_articles = row['expected_articles']
                
                print(f"    📝 Pregunta {idx + 1}/{len(dataset)}: {query[:40]}...")
                
                try:
                    # Evaluar pipeline completo
                    query_metrics = rag_evaluator.evaluate_complete_pipeline(
                        query=query,
                        expected_answer=expected_answer,
                        expected_articles=expected_articles,
                        embedding_model_name=embedding_name,
                        reranking_model_name=None
                    )
                    
                    # Agregar información de la consulta
                    query_metrics['query_id'] = idx
                    combination_results['query_results'].append(query_metrics)
                    
                    # Mostrar métricas principales
                    retrieval_metrics = query_metrics.get('retrieval_metrics', {})
                    quality_scores = query_metrics.get('quality_scores', {})
                    
                    print(f"      📈 Retrieval Recall@5: {retrieval_metrics.get('recall_at_5', 0):.3f}")
                    print(f"      🎯 Calidad Overall: {quality_scores.get('overall_score', 0):.1f}/5")
                    
                except Exception as e:
                    print(f"      ❌ Error en pregunta {idx + 1}: {e}")
                    continue
            
            # Calcular métricas agregadas
            if combination_results['query_results']:
                print(f"\n    📊 Calculando métricas agregadas...")
                aggregated = calculate_rag_aggregated_metrics(combination_results['query_results'])
                combination_results['aggregated_metrics'] = aggregated
                
                # Mostrar resumen
                print(f"    🎯 Resumen:")
                print(f"      - Recall@5 promedio: {aggregated.get('retrieval_recall_at_5', {}).get('mean', 0):.3f}")
                print(f"      - Calidad promedio: {aggregated.get('quality_overall_score', {}).get('mean', 0):.1f}/5")
            
            embedding_results['no_reranking'] = combination_results
        
        else:
            # Evaluar con cada modelo de re-ranking
            for reranking_name in reranking_models.keys():
                print(f"\n  🔄 Re-ranking: {reranking_name}")
                print("  " + "-" * 30)
                
                combination_results = {
                    'embedding_model': embedding_name,
                    'reranking_model': reranking_name,
                    'query_results': [],
                    'aggregated_metrics': {}
                }
                
                # Evaluar cada pregunta
                for idx, row in dataset.iterrows():
                    query = row['question']
                    expected_answer = row['expected_answer']
                    expected_articles = row['expected_articles']
                    
                    print(f"    📝 Pregunta {idx + 1}/{len(dataset)}: {query[:40]}...")
                    
                    try:
                        # Evaluar pipeline completo
                        query_metrics = rag_evaluator.evaluate_complete_pipeline(
                            query=query,
                            expected_answer=expected_answer,
                            expected_articles=expected_articles,
                            embedding_model_name=embedding_name,
                            reranking_model_name=reranking_name
                        )
                        
                        # Agregar información de la consulta
                        query_metrics['query_id'] = idx
                        combination_results['query_results'].append(query_metrics)
                        
                        # Mostrar métricas principales
                        retrieval_metrics = query_metrics.get('retrieval_metrics', {})
                        quality_scores = query_metrics.get('quality_scores', {})
                        
                        print(f"      📈 Retrieval Recall@5: {retrieval_metrics.get('recall_at_5', 0):.3f}")
                        print(f"      🎯 Calidad Overall: {quality_scores.get('overall_score', 0):.1f}/5")
                        
                    except Exception as e:
                        print(f"      ❌ Error en pregunta {idx + 1}: {e}")
                        continue
                
                # Calcular métricas agregadas
                if combination_results['query_results']:
                    print(f"\n    📊 Calculando métricas agregadas...")
                    aggregated = calculate_rag_aggregated_metrics(combination_results['query_results'])
                    combination_results['aggregated_metrics'] = aggregated
                    
                    # Mostrar resumen
                    print(f"    🎯 Resumen:")
                    print(f"      - Recall@5 promedio: {aggregated.get('retrieval_recall_at_5', {}).get('mean', 0):.3f}")
                    print(f"      - Calidad promedio: {aggregated.get('quality_overall_score', {}).get('mean', 0):.1f}/5")
                
                embedding_results[reranking_name] = combination_results
        
        all_results[embedding_name] = embedding_results
    
    print("\n" + "=" * 70)
    print("✅ Evaluación del pipeline RAG completada")
    
    return all_results

def calculate_rag_aggregated_metrics(query_results: List[Dict[str, Any]]) -> Dict[str, Dict[str, float]]:
    """
    Calcula métricas agregadas para evaluación del pipeline RAG
    
    Args:
        query_results: Lista de resultados de consultas individuales
    
    Returns:
        Diccionario con métricas agregadas
    """
    if not query_results:
        return {}
    
    # Extraer métricas de retrieval
    retrieval_metrics = {}
    for result in query_results:
        if 'retrieval_metrics' in result:
            for metric, value in result['retrieval_metrics'].items():
                if f'retrieval_{metric}' not in retrieval_metrics:
                    retrieval_metrics[f'retrieval_{metric}'] = []
                retrieval_metrics[f'retrieval_{metric}'].append(value)
    
    # Extraer métricas de calidad
    quality_metrics = {}
    for result in query_results:
        if 'quality_scores' in result:
            for metric, value in result['quality_scores'].items():
                if metric != 'explanation':  # Excluir explicación
                    if f'quality_{metric}' not in quality_metrics:
                        quality_metrics[f'quality_{metric}'] = []
                    quality_metrics[f'quality_{metric}'].append(value)
    
    # Combinar todas las métricas
    all_metrics = {**retrieval_metrics, **quality_metrics}
    
    # Calcular estadísticas
    aggregated = {}
    for metric_name, values in all_metrics.items():
        if values:
            aggregated[metric_name] = {
                'mean': np.mean(values),
                'std': np.std(values),
                'min': np.min(values),
                'max': np.max(values),
                'median': np.median(values)
            }
    
    return aggregated

# Ejecutar evaluación completa del pipeline RAG
print("🔄 Iniciando evaluación completa del pipeline RAG...")
complete_rag_results = evaluate_complete_rag_pipeline(
    dataset, 
    env_manager.embedding_models, 
    env_manager.reranking_models,
    use_reranking=True
)


In [ ]:
# Visualización de resultados del pipeline RAG completo
def visualize_complete_rag_results(rag_results: Dict[str, Any]):
    """
    Visualiza los resultados de evaluación del pipeline RAG completo
    """
    if not rag_results:
        print("❌ No hay resultados para visualizar")
        return
    
    # Preparar datos para visualización
    combinations = []
    retrieval_data = []
    quality_data = []
    
    for embedding_name, embedding_data in rag_results.items():
        for reranking_name, combination_data in embedding_data.items():
            if combination_data['aggregated_metrics']:
                combination_key = f"{embedding_name}\n+ {reranking_name if reranking_name else 'No Re-ranking'}"
                combinations.append(combination_key)
                
                # Extraer métricas de retrieval
                metrics = combination_data['aggregated_metrics']
                retrieval_data.append({
                    'combination': combination_key,
                    'recall_at_5': metrics.get('retrieval_recall_at_5', {}).get('mean', 0),
                    'precision_at_5': metrics.get('retrieval_precision_at_5', {}).get('mean', 0),
                    'ndcg_at_5': metrics.get('retrieval_ndcg_at_5', {}).get('mean', 0),
                    'mrr': metrics.get('retrieval_mrr', {}).get('mean', 0)
                })
                
                # Extraer métricas de calidad
                quality_data.append({
                    'combination': combination_key,
                    'coherence': metrics.get('quality_coherence', {}).get('mean', 0),
                    'relevance': metrics.get('quality_relevance', {}).get('mean', 0),
                    'completeness': metrics.get('quality_completeness', {}).get('mean', 0),
                    'fidelity': metrics.get('quality_fidelity', {}).get('mean', 0),
                    'conciseness': metrics.get('quality_conciseness', {}).get('mean', 0),
                    'overall_score': metrics.get('quality_overall_score', {}).get('mean', 0)
                })
    
    if not retrieval_data or not quality_data:
        print("❌ No hay datos suficientes para visualizar")
        return
    
    # Crear figura con subplots
    fig, axes = plt.subplots(3, 2, figsize=(16, 18))
    fig.suptitle('Evaluación Completa del Pipeline RAG', fontsize=16, fontweight='bold')
    
    # Colores para cada combinación
    colors = plt.cm.Set3(np.linspace(0, 1, len(combinations)))
    
    # Métricas de retrieval
    retrieval_metrics = ['recall_at_5', 'precision_at_5', 'ndcg_at_5', 'mrr']
    retrieval_titles = ['Recall@5', 'Precision@5', 'nDCG@5', 'MRR']
    
    for idx, (metric, title) in enumerate(zip(retrieval_metrics, retrieval_titles)):
        row = idx // 2
        col = idx % 2
        
        values = [data[metric] for data in retrieval_data]
        combination_labels = [data['combination'] for data in retrieval_data]
        
        bars = axes[row, col].bar(range(len(values)), values, color=colors, alpha=0.7)
        axes[row, col].set_title(f'Retrieval - {title}')
        axes[row, col].set_ylabel('Score')
        axes[row, col].set_ylim(0, 1)
        axes[row, col].set_xticks(range(len(combination_labels)))
        axes[row, col].set_xticklabels(combination_labels, rotation=45, ha='right')
        
        # Agregar valores en las barras
        for bar, value in zip(bars, values):
            height = bar.get_height()
            axes[row, col].text(bar.get_x() + bar.get_width()/2., height + 0.01,
                              f'{value:.3f}', ha='center', va='bottom', fontsize=8)
    
    # Métricas de calidad
    quality_metrics = ['coherence', 'relevance', 'completeness', 'fidelity', 'conciseness', 'overall_score']
    quality_titles = ['Coherencia', 'Relevancia', 'Completitud', 'Fidelidad', 'Concisión', 'Score General']
    
    for idx, (metric, title) in enumerate(zip(quality_metrics, quality_titles)):
        row = 2
        col = idx % 2
        
        values = [data[metric] for data in quality_data]
        combination_labels = [data['combination'] for data in quality_data]
        
        bars = axes[row, col].bar(range(len(values)), values, color=colors, alpha=0.7)
        axes[row, col].set_title(f'Calidad - {title}')
        axes[row, col].set_ylabel('Score (1-5)')
        axes[row, col].set_ylim(0, 5)
        axes[row, col].set_xticks(range(len(combination_labels)))
        axes[row, col].set_xticklabels(combination_labels, rotation=45, ha='right')
        
        # Agregar valores en las barras
        for bar, value in zip(bars, values):
            height = bar.get_height()
            axes[row, col].text(bar.get_x() + bar.get_width()/2., height + 0.05,
                              f'{value:.1f}', ha='center', va='bottom', fontsize=8)
    
    plt.tight_layout()
    plt.show()
    
    # Crear tabla comparativa
    print("\n📊 Tabla Comparativa del Pipeline RAG:")
    print("=" * 120)
    
    # Combinar datos para la tabla
    comparison_data = []
    for i, retrieval in enumerate(retrieval_data):
        quality = quality_data[i]
        row = {
            'Combinación': retrieval['combination'],
            'Recall@5': f"{retrieval['recall_at_5']:.3f}",
            'Precision@5': f"{retrieval['precision_at_5']:.3f}",
            'nDCG@5': f"{retrieval['ndcg_at_5']:.3f}",
            'MRR': f"{retrieval['mrr']:.3f}",
            'Coherencia': f"{quality['coherence']:.1f}",
            'Relevancia': f"{quality['relevance']:.1f}",
            'Completitud': f"{quality['completeness']:.1f}",
            'Fidelidad': f"{quality['fidelity']:.1f}",
            'Concisión': f"{quality['conciseness']:.1f}",
            'Score General': f"{quality['overall_score']:.1f}"
        }
        comparison_data.append(row)
    
    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))
    
    # Identificar mejores combinaciones
    print("\n🏆 Mejores Combinaciones:")
    print("-" * 50)
    
    # Mejor en retrieval
    best_retrieval_idx = max(range(len(retrieval_data)), 
                           key=lambda i: retrieval_data[i]['recall_at_5'])
    print(f"  Mejor Retrieval: {retrieval_data[best_retrieval_idx]['combination']} "
          f"(Recall@5: {retrieval_data[best_retrieval_idx]['recall_at_5']:.3f})")
    
    # Mejor en calidad
    best_quality_idx = max(range(len(quality_data)), 
                          key=lambda i: quality_data[i]['overall_score'])
    print(f"  Mejor Calidad: {quality_data[best_quality_idx]['combination']} "
          f"(Score: {quality_data[best_quality_idx]['overall_score']:.1f}/5)")

def analyze_complete_rag_performance(rag_results: Dict[str, Any]):
    """
    Analiza el rendimiento del pipeline RAG completo en detalle
    """
    print("\n🔍 Análisis Detallado del Pipeline RAG:")
    print("=" * 60)
    
    for embedding_name, embedding_data in rag_results.items():
        print(f"\n🧠 Embedding: {embedding_name}")
        print("-" * 40)
        
        for reranking_name, combination_data in embedding_data.items():
            print(f"\n  🔄 Re-ranking: {reranking_name if reranking_name else 'No Re-ranking'}")
            print("  " + "-" * 30)
            
            aggregated = combination_data['aggregated_metrics']
            query_results = combination_data['query_results']
            
            if not aggregated:
                print("    ❌ No hay métricas disponibles")
                continue
            
            # Métricas de retrieval
            print(f"  📈 Métricas de Retrieval:")
            for metric in ['retrieval_recall_at_5', 'retrieval_precision_at_5', 
                          'retrieval_ndcg_at_5', 'retrieval_mrr']:
                if metric in aggregated:
                    mean_val = aggregated[metric]['mean']
                    std_val = aggregated[metric]['std']
                    print(f"    {metric.replace('retrieval_', '').replace('_at_5', '@5').title()}: {mean_val:.3f} ± {std_val:.3f}")
            
            # Métricas de calidad
            print(f"  🎯 Métricas de Calidad:")
            for metric in ['quality_coherence', 'quality_relevance', 'quality_completeness', 
                          'quality_fidelity', 'quality_conciseness', 'quality_overall_score']:
                if metric in aggregated:
                    mean_val = aggregated[metric]['mean']
                    std_val = aggregated[metric]['std']
                    print(f"    {metric.replace('quality_', '').title()}: {mean_val:.1f} ± {std_val:.1f}")
            
            # Análisis de respuestas
            if query_results:
                print(f"  📝 Análisis de Respuestas:")
                
                # Mejores respuestas (mayor calidad general)
                best_responses = sorted(query_results, 
                                      key=lambda x: x.get('quality_scores', {}).get('overall_score', 0), 
                                      reverse=True)[:3]
                print(f"    Mejores respuestas (Calidad General):")
                for i, response in enumerate(best_responses, 1):
                    quality = response.get('quality_scores', {})
                    overall_score = quality.get('overall_score', 0)
                    query_text = response.get('query', '')[:50]
                    print(f"      {i}. {query_text}... (Score: {overall_score:.1f}/5)")
                
                # Peores respuestas
                worst_responses = sorted(query_results, 
                                       key=lambda x: x.get('quality_scores', {}).get('overall_score', 0))[:3]
                print(f"    Peores respuestas (Calidad General):")
                for i, response in enumerate(worst_responses, 1):
                    quality = response.get('quality_scores', {})
                    overall_score = quality.get('overall_score', 0)
                    query_text = response.get('query', '')[:50]
                    print(f"      {i}. {query_text}... (Score: {overall_score:.1f}/5)")

# Ejecutar visualizaciones y análisis
visualize_complete_rag_results(complete_rag_results)
analyze_complete_rag_performance(complete_rag_results)


In [ ]:
# Guardar resultados de evaluación del pipeline RAG completo
def save_complete_rag_evaluation_results(rag_results: Dict[str, Any], 
                                        evaluation_name: str = "complete_rag_evaluation") -> str:
    """
    Guarda los resultados de evaluación del pipeline RAG completo
    
    Args:
        rag_results: Resultados de la evaluación
        evaluation_name: Nombre de la evaluación
    
    Returns:
        Ruta del archivo guardado
    """
    # Preparar configuración
    config = {
        'embedding_models': list(EMBEDDING_MODELS.keys()),
        'reranking_models': list(RERANKING_MODELS.keys()),
        'retrieval_metrics': RETRIEVAL_METRICS,
        'llm_evaluation_criteria': LLM_EVALUATION_CRITERIA,
        'qdrant_config': QDRANT_CONFIG,
        'llm_config': LLM_CONFIG,
        'dataset_size': len(dataset)
    }
    
    # Preparar metadatos
    metadata = {
        'evaluation_type': 'complete_rag',
        'description': 'Evaluación completa del pipeline RAG incluyendo generación y evaluación con LLM',
        'timestamp': datetime.now().isoformat(),
        'total_queries': len(dataset),
        'total_combinations': sum(len(embedding_data) for embedding_data in rag_results.values())
    }
    
    # Guardar evaluación
    filepath = eval_manager.save_evaluation(
        evaluation_name=evaluation_name,
        config=config,
        results={'complete_rag_metrics': rag_results},
        metadata=metadata
    )
    
    print(f"💾 Resultados de evaluación del pipeline RAG completo guardados en: {filepath}")
    return filepath

# Guardar resultados
complete_rag_filepath = save_complete_rag_evaluation_results(complete_rag_results)

# Resumen final de la evaluación del pipeline RAG completo
print("\n" + "=" * 80)
print("📋 RESUMEN DE EVALUACIÓN DEL PIPELINE RAG COMPLETO")
print("=" * 80)

if complete_rag_results:
    # Encontrar la mejor combinación general
    best_combination = None
    best_overall_score = -1
    
    for embedding_name, embedding_data in complete_rag_results.items():
        for reranking_name, combination_data in embedding_data.items():
            aggregated = combination_data['aggregated_metrics']
            if aggregated:
                # Calcular score combinado (retrieval + calidad)
                retrieval_score = aggregated.get('retrieval_recall_at_5', {}).get('mean', 0)
                quality_score = aggregated.get('quality_overall_score', {}).get('mean', 0)
                
                # Score combinado (peso 40% retrieval, 60% calidad)
                combined_score = (0.4 * retrieval_score) + (0.6 * (quality_score / 5))
                
                if combined_score > best_overall_score:
                    best_overall_score = combined_score
                    best_combination = f"{embedding_name} + {reranking_name if reranking_name else 'No Re-ranking'}"
    
    if best_combination:
        print(f"🏆 Mejor combinación general: {best_combination}")
        print(f"   Score combinado: {best_overall_score:.3f}")
    
    # Estadísticas por combinación
    print(f"\n📊 Estadísticas por combinación:")
    for embedding_name, embedding_data in complete_rag_results.items():
        print(f"\n  🧠 Embedding: {embedding_name}")
        for reranking_name, combination_data in embedding_data.items():
            aggregated = combination_data['aggregated_metrics']
            if aggregated:
                # Métricas de retrieval
                recall = aggregated.get('retrieval_recall_at_5', {}).get('mean', 0)
                precision = aggregated.get('retrieval_precision_at_5', {}).get('mean', 0)
                ndcg = aggregated.get('retrieval_ndcg_at_5', {}).get('mean', 0)
                mrr = aggregated.get('retrieval_mrr', {}).get('mean', 0)
                
                # Métricas de calidad
                coherence = aggregated.get('quality_coherence', {}).get('mean', 0)
                relevance = aggregated.get('quality_relevance', {}).get('mean', 0)
                completeness = aggregated.get('quality_completeness', {}).get('mean', 0)
                fidelity = aggregated.get('quality_fidelity', {}).get('mean', 0)
                conciseness = aggregated.get('quality_conciseness', {}).get('mean', 0)
                overall_quality = aggregated.get('quality_overall_score', {}).get('mean', 0)
                
                print(f"    🔄 {reranking_name if reranking_name else 'No Re-ranking'}:")
                print(f"      📈 Retrieval:")
                print(f"        - Recall@5: {recall:.3f}")
                print(f"        - Precision@5: {precision:.3f}")
                print(f"        - nDCG@5: {ndcg:.3f}")
                print(f"        - MRR: {mrr:.3f}")
                print(f"      🎯 Calidad:")
                print(f"        - Coherencia: {coherence:.1f}/5")
                print(f"        - Relevancia: {relevance:.1f}/5")
                print(f"        - Completitud: {completeness:.1f}/5")
                print(f"        - Fidelidad: {fidelity:.1f}/5")
                print(f"        - Concisión: {conciseness:.1f}/5")
                print(f"        - Score General: {overall_quality:.1f}/5")
    
    # Análisis de correlación entre retrieval y calidad
    print(f"\n📈 Análisis de Correlación:")
    print("-" * 50)
    
    retrieval_scores = []
    quality_scores = []
    
    for embedding_data in complete_rag_results.values():
        for combination_data in embedding_data.values():
            aggregated = combination_data['aggregated_metrics']
            if aggregated:
                retrieval_score = aggregated.get('retrieval_recall_at_5', {}).get('mean', 0)
                quality_score = aggregated.get('quality_overall_score', {}).get('mean', 0)
                if retrieval_score > 0 and quality_score > 0:
                    retrieval_scores.append(retrieval_score)
                    quality_scores.append(quality_score)
    
    if len(retrieval_scores) > 1:
        correlation = np.corrcoef(retrieval_scores, quality_scores)[0, 1]
        print(f"  - Correlación Retrieval-Calidad: {correlation:.3f}")
        
        if correlation > 0.5:
            print("  ✅ Fuerte correlación positiva entre retrieval y calidad")
        elif correlation > 0.2:
            print("  ⚠️  Correlación moderada entre retrieval y calidad")
        else:
            print("  ❌ Correlación débil entre retrieval y calidad")
    
    print(f"\n✅ Evaluación del pipeline RAG completo finalizada exitosamente")
    print(f"📁 Resultados guardados en: {complete_rag_filepath}")
else:
    print("❌ No se pudieron obtener resultados de evaluación del pipeline RAG completo")


# 7. Análisis Comparativo Global

## Objetivo del Análisis

Esta sección realiza un análisis comparativo integral de todos los resultados obtenidos en las evaluaciones anteriores, combinando métricas de retrieval, re-ranking y calidad de respuesta para identificar patrones, tendencias y recomendaciones finales.

### 🎯 Proceso de Análisis Global

1. **Consolidación de Resultados**: Integrar datos de todas las evaluaciones
2. **Análisis Comparativo**: Comparar rendimiento entre modelos y combinaciones
3. **Identificación de Patrones**: Detectar tendencias y correlaciones
4. **Ranking Global**: Clasificar combinaciones por rendimiento general
5. **Recomendaciones**: Sugerir mejores configuraciones y mejoras

### 📊 Dimensiones de Análisis

#### Análisis por Componente
- **Embeddings**: Rendimiento de diferentes modelos de embedding
- **Re-ranking**: Efectividad de modelos cross-encoder
- **LLM**: Calidad de generación y evaluación
- **Pipeline Completo**: Rendimiento end-to-end

#### Análisis por Métrica
- **Retrieval**: Recall, Precision, nDCG, MRR
- **Calidad**: Coherencia, Relevancia, Completitud, Fidelidad, Concisión
- **Eficiencia**: Tiempo de procesamiento, uso de recursos
- **Robustez**: Consistencia entre consultas

#### Análisis por Combinación
- **Mejores Combinaciones**: Top performers por criterio
- **Trade-offs**: Balance entre diferentes métricas
- **Escalabilidad**: Rendimiento con diferentes tamaños de dataset
- **Estabilidad**: Consistencia de resultados


In [ ]:
# Clase para análisis comparativo global
class GlobalComparativeAnalyzer:
    """
    Realiza análisis comparativo global de todos los resultados de evaluación
    """
    
    def __init__(self, retrieval_results: Dict[str, Any], 
                 reranking_results: Dict[str, Any], 
                 complete_rag_results: Dict[str, Any]):
        self.retrieval_results = retrieval_results
        self.reranking_results = reranking_results
        self.complete_rag_results = complete_rag_results
        self.analysis_results = {}
    
    def consolidate_all_results(self) -> Dict[str, Any]:
        """
        Consolida todos los resultados de las evaluaciones
        
        Returns:
            Diccionario con resultados consolidados
        """
        print("🔄 Consolidando resultados de todas las evaluaciones...")
        
        consolidated = {
            'retrieval_only': self._consolidate_retrieval_results(),
            'reranking_improvements': self._consolidate_reranking_results(),
            'complete_pipeline': self._consolidate_complete_rag_results(),
            'global_rankings': {},
            'insights': {}
        }
        
        # Generar rankings globales
        consolidated['global_rankings'] = self._generate_global_rankings(consolidated)
        
        # Generar insights
        consolidated['insights'] = self._generate_insights(consolidated)
        
        print("✅ Consolidación completada")
        return consolidated
    
    def _consolidate_retrieval_results(self) -> Dict[str, Any]:
        """
        Consolida resultados de evaluación de retrievers
        """
        if not self.retrieval_results:
            return {}
        
        consolidated = {}
        
        for model_name, model_data in self.retrieval_results.items():
            if 'aggregated_metrics' in model_data:
                metrics = model_data['aggregated_metrics']
                consolidated[model_name] = {
                    'recall_at_5': metrics.get('recall_at_5', {}).get('mean', 0),
                    'precision_at_5': metrics.get('precision_at_5', {}).get('mean', 0),
                    'ndcg_at_5': metrics.get('ndcg_at_5', {}).get('mean', 0),
                    'mrr': metrics.get('mrr', {}).get('mean', 0),
                    'total_queries': len(model_data.get('query_results', []))
                }
        
        return consolidated
    
    def _consolidate_reranking_results(self) -> Dict[str, Any]:
        """
        Consolida resultados de evaluación de re-ranking
        """
        if not self.reranking_results:
            return {}
        
        consolidated = {}
        
        for embedding_name, embedding_data in self.reranking_results.items():
            for reranking_name, combination_data in embedding_data.items():
                if 'aggregated_metrics' in combination_data:
                    metrics = combination_data['aggregated_metrics']
                    key = f"{embedding_name}+{reranking_name}"
                    consolidated[key] = {
                        'embedding_model': embedding_name,
                        'reranking_model': reranking_name,
                        'recall_improvement': metrics.get('recall_improvement_at_5', {}).get('mean', 0),
                        'precision_improvement': metrics.get('precision_improvement_at_5', {}).get('mean', 0),
                        'ndcg_improvement': metrics.get('ndcg_improvement_at_5', {}).get('mean', 0),
                        'mrr_improvement': metrics.get('mrr_improvement', {}).get('mean', 0),
                        'total_queries': len(combination_data.get('query_results', []))
                    }
        
        return consolidated
    
    def _consolidate_complete_rag_results(self) -> Dict[str, Any]:
        """
        Consolida resultados de evaluación del pipeline completo
        """
        if not self.complete_rag_results:
            return {}
        
        consolidated = {}
        
        for embedding_name, embedding_data in self.complete_rag_results.items():
            for reranking_name, combination_data in embedding_data.items():
                if 'aggregated_metrics' in combination_data:
                    metrics = combination_data['aggregated_metrics']
                    key = f"{embedding_name}+{reranking_name if reranking_name else 'NoReranking'}"
                    consolidated[key] = {
                        'embedding_model': embedding_name,
                        'reranking_model': reranking_name,
                        'retrieval_recall_at_5': metrics.get('retrieval_recall_at_5', {}).get('mean', 0),
                        'retrieval_precision_at_5': metrics.get('retrieval_precision_at_5', {}).get('mean', 0),
                        'retrieval_ndcg_at_5': metrics.get('retrieval_ndcg_at_5', {}).get('mean', 0),
                        'retrieval_mrr': metrics.get('retrieval_mrr', {}).get('mean', 0),
                        'quality_coherence': metrics.get('quality_coherence', {}).get('mean', 0),
                        'quality_relevance': metrics.get('quality_relevance', {}).get('mean', 0),
                        'quality_completeness': metrics.get('quality_completeness', {}).get('mean', 0),
                        'quality_fidelity': metrics.get('quality_fidelity', {}).get('mean', 0),
                        'quality_conciseness': metrics.get('quality_conciseness', {}).get('mean', 0),
                        'quality_overall_score': metrics.get('quality_overall_score', {}).get('mean', 0),
                        'total_queries': len(combination_data.get('query_results', []))
                    }
        
        return consolidated
    
    def _generate_global_rankings(self, consolidated: Dict[str, Any]) -> Dict[str, List[str]]:
        """
        Genera rankings globales por diferentes criterios
        """
        rankings = {}
        
        # Ranking por retrieval (solo embeddings)
        if 'retrieval_only' in consolidated:
            retrieval_data = consolidated['retrieval_only']
            rankings['best_retrieval'] = sorted(
                retrieval_data.keys(),
                key=lambda x: retrieval_data[x]['recall_at_5'],
                reverse=True
            )
        
        # Ranking por mejoras de re-ranking
        if 'reranking_improvements' in consolidated:
            reranking_data = consolidated['reranking_improvements']
            rankings['best_reranking_improvement'] = sorted(
                reranking_data.keys(),
                key=lambda x: reranking_data[x]['ndcg_improvement'],
                reverse=True
            )
        
        # Ranking por pipeline completo (score combinado)
        if 'complete_pipeline' in consolidated:
            pipeline_data = consolidated['complete_pipeline']
            rankings['best_complete_pipeline'] = sorted(
                pipeline_data.keys(),
                key=lambda x: self._calculate_combined_score(pipeline_data[x]),
                reverse=True
            )
        
        # Ranking por calidad de respuesta
        if 'complete_pipeline' in consolidated:
            pipeline_data = consolidated['complete_pipeline']
            rankings['best_quality'] = sorted(
                pipeline_data.keys(),
                key=lambda x: pipeline_data[x]['quality_overall_score'],
                reverse=True
            )
        
        return rankings
    
    def _calculate_combined_score(self, metrics: Dict[str, Any]) -> float:
        """
        Calcula score combinado (40% retrieval + 60% calidad)
        """
        retrieval_score = metrics.get('retrieval_recall_at_5', 0)
        quality_score = metrics.get('quality_overall_score', 0) / 5  # Normalizar a 0-1
        
        return (0.4 * retrieval_score) + (0.6 * quality_score)
    
    def _generate_insights(self, consolidated: Dict[str, Any]) -> Dict[str, Any]:
        """
        Genera insights y análisis de los resultados
        """
        insights = {
            'retrieval_insights': self._analyze_retrieval_patterns(consolidated),
            'reranking_insights': self._analyze_reranking_patterns(consolidated),
            'quality_insights': self._analyze_quality_patterns(consolidated),
            'correlation_insights': self._analyze_correlations(consolidated),
            'recommendations': self._generate_recommendations(consolidated)
        }
        
        return insights
    
    def _analyze_retrieval_patterns(self, consolidated: Dict[str, Any]) -> Dict[str, Any]:
        """
        Analiza patrones en los resultados de retrieval
        """
        if 'retrieval_only' not in consolidated:
            return {}
        
        retrieval_data = consolidated['retrieval_only']
        
        # Encontrar mejor y peor modelo
        best_model = max(retrieval_data.keys(), key=lambda x: retrieval_data[x]['recall_at_5'])
        worst_model = min(retrieval_data.keys(), key=lambda x: retrieval_data[x]['recall_at_5'])
        
        # Calcular estadísticas
        recall_scores = [data['recall_at_5'] for data in retrieval_data.values()]
        precision_scores = [data['precision_at_5'] for data in retrieval_data.values()]
        ndcg_scores = [data['ndcg_at_5'] for data in retrieval_data.values()]
        
        return {
            'best_model': best_model,
            'worst_model': worst_model,
            'recall_range': (min(recall_scores), max(recall_scores)),
            'precision_range': (min(precision_scores), max(precision_scores)),
            'ndcg_range': (min(ndcg_scores), max(ndcg_scores)),
            'recall_std': np.std(recall_scores),
            'precision_std': np.std(precision_scores),
            'ndcg_std': np.std(ndcg_scores)
        }
    
    def _analyze_reranking_patterns(self, consolidated: Dict[str, Any]) -> Dict[str, Any]:
        """
        Analiza patrones en los resultados de re-ranking
        """
        if 'reranking_improvements' not in consolidated:
            return {}
        
        reranking_data = consolidated['reranking_improvements']
        
        # Calcular mejoras promedio por modelo de re-ranking
        reranking_models = {}
        for key, data in reranking_data.items():
            model_name = data['reranking_model']
            if model_name not in reranking_models:
                reranking_models[model_name] = []
            reranking_models[model_name].append(data['ndcg_improvement'])
        
        # Calcular estadísticas por modelo
        model_stats = {}
        for model_name, improvements in reranking_models.items():
            model_stats[model_name] = {
                'mean_improvement': np.mean(improvements),
                'std_improvement': np.std(improvements),
                'positive_improvements': sum(1 for x in improvements if x > 0),
                'total_combinations': len(improvements)
            }
        
        # Encontrar mejor modelo de re-ranking
        best_reranking_model = max(model_stats.keys(), 
                                 key=lambda x: model_stats[x]['mean_improvement'])
        
        return {
            'best_reranking_model': best_reranking_model,
            'model_stats': model_stats,
            'overall_effectiveness': sum(1 for data in reranking_data.values() 
                                       if data['ndcg_improvement'] > 0) / len(reranking_data)
        }
    
    def _analyze_quality_patterns(self, consolidated: Dict[str, Any]) -> Dict[str, Any]:
        """
        Analiza patrones en los resultados de calidad
        """
        if 'complete_pipeline' not in consolidated:
            return {}
        
        pipeline_data = consolidated['complete_pipeline']
        
        # Calcular estadísticas de calidad
        quality_metrics = ['quality_coherence', 'quality_relevance', 'quality_completeness', 
                          'quality_fidelity', 'quality_conciseness', 'quality_overall_score']
        
        quality_stats = {}
        for metric in quality_metrics:
            values = [data[metric] for data in pipeline_data.values() if metric in data]
            if values:
                quality_stats[metric] = {
                    'mean': np.mean(values),
                    'std': np.std(values),
                    'min': np.min(values),
                    'max': np.max(values)
                }
        
        # Encontrar mejor combinación por calidad
        best_quality_combo = max(pipeline_data.keys(), 
                               key=lambda x: pipeline_data[x]['quality_overall_score'])
        
        return {
            'best_quality_combo': best_quality_combo,
            'quality_stats': quality_stats,
            'overall_quality_mean': quality_stats.get('quality_overall_score', {}).get('mean', 0)
        }
    
    def _analyze_correlations(self, consolidated: Dict[str, Any]) -> Dict[str, Any]:
        """
        Analiza correlaciones entre diferentes métricas
        """
        if 'complete_pipeline' not in consolidated:
            return {}
        
        pipeline_data = consolidated['complete_pipeline']
        
        # Extraer métricas para correlación
        retrieval_scores = []
        quality_scores = []
        coherence_scores = []
        relevance_scores = []
        
        for data in pipeline_data.values():
            retrieval_scores.append(data['retrieval_recall_at_5'])
            quality_scores.append(data['quality_overall_score'])
            coherence_scores.append(data['quality_coherence'])
            relevance_scores.append(data['quality_relevance'])
        
        # Calcular correlaciones
        correlations = {}
        if len(retrieval_scores) > 1:
            correlations['retrieval_quality'] = np.corrcoef(retrieval_scores, quality_scores)[0, 1]
            correlations['retrieval_coherence'] = np.corrcoef(retrieval_scores, coherence_scores)[0, 1]
            correlations['retrieval_relevance'] = np.corrcoef(retrieval_scores, relevance_scores)[0, 1]
            correlations['coherence_relevance'] = np.corrcoef(coherence_scores, relevance_scores)[0, 1]
        
        return correlations
    
    def _generate_recommendations(self, consolidated: Dict[str, Any]) -> Dict[str, Any]:
        """
        Genera recomendaciones basadas en el análisis
        """
        recommendations = {
            'best_overall_config': None,
            'best_retrieval_config': None,
            'best_quality_config': None,
            'reranking_recommendation': None,
            'improvement_suggestions': []
        }
        
        # Mejor configuración general
        if 'global_rankings' in consolidated and 'best_complete_pipeline' in consolidated['global_rankings']:
            best_config = consolidated['global_rankings']['best_complete_pipeline'][0]
            recommendations['best_overall_config'] = best_config
        
        # Mejor configuración de retrieval
        if 'global_rankings' in consolidated and 'best_retrieval' in consolidated['global_rankings']:
            best_retrieval = consolidated['global_rankings']['best_retrieval'][0]
            recommendations['best_retrieval_config'] = best_retrieval
        
        # Mejor configuración de calidad
        if 'global_rankings' in consolidated and 'best_quality' in consolidated['global_rankings']:
            best_quality = consolidated['global_rankings']['best_quality'][0]
            recommendations['best_quality_config'] = best_quality
        
        # Recomendación de re-ranking
        if 'reranking_insights' in consolidated['insights']:
            reranking_insights = consolidated['insights']['reranking_insights']
            if reranking_insights.get('overall_effectiveness', 0) > 0.5:
                recommendations['reranking_recommendation'] = f"Usar re-ranking con {reranking_insights['best_reranking_model']}"
            else:
                recommendations['reranking_recommendation'] = "Re-ranking no muestra mejoras significativas"
        
        # Sugerencias de mejora
        suggestions = []
        
        # Analizar correlaciones
        if 'correlation_insights' in consolidated['insights']:
            correlations = consolidated['insights']['correlation_insights']
            if correlations.get('retrieval_quality', 0) < 0.3:
                suggestions.append("Mejorar la calidad de retrieval para mejorar respuestas")
            if correlations.get('coherence_relevance', 0) < 0.5:
                suggestions.append("Trabajar en la coherencia y relevancia de las respuestas")
        
        recommendations['improvement_suggestions'] = suggestions
        
        return recommendations

# Inicializar analizador global
global_analyzer = GlobalComparativeAnalyzer(
    retrieval_results=retrieval_results,
    reranking_results=reranking_results,
    complete_rag_results=complete_rag_results
)

print("✅ Analizador comparativo global inicializado")


In [ ]:
# Ejecutar análisis comparativo global
print("🔄 Iniciando análisis comparativo global...")
global_analysis = global_analyzer.consolidate_all_results()

# Visualización del análisis global
def visualize_global_analysis(global_analysis: Dict[str, Any]):
    """
    Visualiza el análisis comparativo global
    """
    if not global_analysis:
        print("❌ No hay datos para visualizar")
        return
    
    # Crear figura con múltiples subplots
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    fig.suptitle('Análisis Comparativo Global - Pipeline RAG', fontsize=16, fontweight='bold')
    
    # 1. Ranking de modelos de embedding (retrieval)
    if 'retrieval_only' in global_analysis and global_analysis['retrieval_only']:
        retrieval_data = global_analysis['retrieval_only']
        models = list(retrieval_data.keys())
        recall_scores = [retrieval_data[model]['recall_at_5'] for model in models]
        
        axes[0, 0].bar(models, recall_scores, color='skyblue', alpha=0.7)
        axes[0, 0].set_title('Ranking de Embeddings (Recall@5)')
        axes[0, 0].set_ylabel('Recall@5')
        axes[0, 0].tick_params(axis='x', rotation=45)
        
        # Agregar valores en las barras
        for i, v in enumerate(recall_scores):
            axes[0, 0].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
    
    # 2. Mejoras de re-ranking
    if 'reranking_improvements' in global_analysis and global_analysis['reranking_improvements']:
        reranking_data = global_analysis['reranking_improvements']
        combinations = list(reranking_data.keys())
        improvements = [reranking_data[combo]['ndcg_improvement'] for combo in combinations]
        
        axes[0, 1].bar(range(len(combinations)), improvements, color='lightgreen', alpha=0.7)
        axes[0, 1].set_title('Mejoras de Re-ranking (nDCG@5)')
        axes[0, 1].set_ylabel('Mejora nDCG@5')
        axes[0, 1].set_xticks(range(len(combinations)))
        axes[0, 1].set_xticklabels(combinations, rotation=45, ha='right')
        axes[0, 1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
        
        # Agregar valores en las barras
        for i, v in enumerate(improvements):
            axes[0, 1].text(i, v + (0.001 if v >= 0 else -0.003), f'{v:+.3f}', 
                           ha='center', va='bottom' if v >= 0 else 'top', fontsize=8)
    
    # 3. Calidad de respuestas
    if 'complete_pipeline' in global_analysis and global_analysis['complete_pipeline']:
        pipeline_data = global_analysis['complete_pipeline']
        combinations = list(pipeline_data.keys())
        quality_scores = [pipeline_data[combo]['quality_overall_score'] for combo in combinations]
        
        axes[0, 2].bar(range(len(combinations)), quality_scores, color='orange', alpha=0.7)
        axes[0, 2].set_title('Calidad de Respuestas (Score General)')
        axes[0, 2].set_ylabel('Score (1-5)')
        axes[0, 2].set_xticks(range(len(combinations)))
        axes[0, 2].set_xticklabels(combinations, rotation=45, ha='right')
        axes[0, 2].set_ylim(0, 5)
        
        # Agregar valores en las barras
        for i, v in enumerate(quality_scores):
            axes[0, 2].text(i, v + 0.05, f'{v:.1f}', ha='center', va='bottom')
    
    # 4. Score combinado (retrieval + calidad)
    if 'complete_pipeline' in global_analysis and global_analysis['complete_pipeline']:
        pipeline_data = global_analysis['complete_pipeline']
        combinations = list(pipeline_data.keys())
        combined_scores = [global_analyzer._calculate_combined_score(pipeline_data[combo]) for combo in combinations]
        
        axes[1, 0].bar(range(len(combinations)), combined_scores, color='purple', alpha=0.7)
        axes[1, 0].set_title('Score Combinado (40% Retrieval + 60% Calidad)')
        axes[1, 0].set_ylabel('Score Combinado')
        axes[1, 0].set_xticks(range(len(combinations)))
        axes[1, 0].set_xticklabels(combinations, rotation=45, ha='right')
        
        # Agregar valores en las barras
        for i, v in enumerate(combined_scores):
            axes[1, 0].text(i, v + 0.01, f'{v:.3f}', ha='center', va='bottom')
    
    # 5. Correlación Retrieval vs Calidad
    if 'complete_pipeline' in global_analysis and global_analysis['complete_pipeline']:
        pipeline_data = global_analysis['complete_pipeline']
        retrieval_scores = [pipeline_data[combo]['retrieval_recall_at_5'] for combo in pipeline_data.keys()]
        quality_scores = [pipeline_data[combo]['quality_overall_score'] for combo in pipeline_data.keys()]
        
        axes[1, 1].scatter(retrieval_scores, quality_scores, alpha=0.7, s=100)
        axes[1, 1].set_title('Correlación Retrieval vs Calidad')
        axes[1, 1].set_xlabel('Recall@5')
        axes[1, 1].set_ylabel('Calidad General')
        
        # Calcular y mostrar correlación
        if len(retrieval_scores) > 1:
            correlation = np.corrcoef(retrieval_scores, quality_scores)[0, 1]
            axes[1, 1].text(0.05, 0.95, f'Correlación: {correlation:.3f}', 
                           transform=axes[1, 1].transAxes, fontsize=12, 
                           bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.8))
    
    # 6. Distribución de métricas de calidad
    if 'complete_pipeline' in global_analysis and global_analysis['complete_pipeline']:
        pipeline_data = global_analysis['complete_pipeline']
        quality_metrics = ['quality_coherence', 'quality_relevance', 'quality_completeness', 
                          'quality_fidelity', 'quality_conciseness']
        metric_names = ['Coherencia', 'Relevancia', 'Completitud', 'Fidelidad', 'Concisión']
        
        # Calcular promedios por métrica
        metric_means = []
        for metric in quality_metrics:
            values = [pipeline_data[combo][metric] for combo in pipeline_data.keys() if metric in pipeline_data[combo]]
            metric_means.append(np.mean(values) if values else 0)
        
        axes[1, 2].bar(metric_names, metric_means, color='lightcoral', alpha=0.7)
        axes[1, 2].set_title('Distribución de Métricas de Calidad')
        axes[1, 2].set_ylabel('Score Promedio (1-5)')
        axes[1, 2].set_ylim(0, 5)
        axes[1, 2].tick_params(axis='x', rotation=45)
        
        # Agregar valores en las barras
        for i, v in enumerate(metric_means):
            axes[1, 2].text(i, v + 0.05, f'{v:.1f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

# Ejecutar visualización
visualize_global_analysis(global_analysis)


In [ ]:
# Análisis detallado y reporte final
def generate_comprehensive_report(global_analysis: Dict[str, Any]):
    """
    Genera un reporte comprensivo del análisis global
    """
    print("\n" + "=" * 80)
    print("📊 REPORTE COMPREHENSIVO - ANÁLISIS COMPARATIVO GLOBAL")
    print("=" * 80)
    
    # 1. Rankings Globales
    print("\n🏆 RANKINGS GLOBALES")
    print("-" * 50)
    
    if 'global_rankings' in global_analysis:
        rankings = global_analysis['global_rankings']
        
        # Mejor retrieval
        if 'best_retrieval' in rankings and rankings['best_retrieval']:
            print(f"🥇 Mejor Modelo de Embedding (Retrieval): {rankings['best_retrieval'][0]}")
            if len(rankings['best_retrieval']) > 1:
                print(f"   Top 3: {', '.join(rankings['best_retrieval'][:3])}")
        
        # Mejor re-ranking
        if 'best_reranking_improvement' in rankings and rankings['best_reranking_improvement']:
            print(f"🥇 Mejor Mejora de Re-ranking: {rankings['best_reranking_improvement'][0]}")
            if len(rankings['best_reranking_improvement']) > 1:
                print(f"   Top 3: {', '.join(rankings['best_reranking_improvement'][:3])}")
        
        # Mejor pipeline completo
        if 'best_complete_pipeline' in rankings and rankings['best_complete_pipeline']:
            print(f"🥇 Mejor Pipeline Completo: {rankings['best_complete_pipeline'][0]}")
            if len(rankings['best_complete_pipeline']) > 1:
                print(f"   Top 3: {', '.join(rankings['best_complete_pipeline'][:3])}")
        
        # Mejor calidad
        if 'best_quality' in rankings and rankings['best_quality']:
            print(f"🥇 Mejor Calidad de Respuesta: {rankings['best_quality'][0]}")
            if len(rankings['best_quality']) > 1:
                print(f"   Top 3: {', '.join(rankings['best_quality'][:3])}")
    
    # 2. Insights por Componente
    print("\n🔍 INSIGHTS POR COMPONENTE")
    print("-" * 50)
    
    if 'insights' in global_analysis:
        insights = global_analysis['insights']
        
        # Insights de retrieval
        if 'retrieval_insights' in insights and insights['retrieval_insights']:
            retrieval_insights = insights['retrieval_insights']
            print(f"\n📈 Retrieval:")
            print(f"   - Mejor modelo: {retrieval_insights.get('best_model', 'N/A')}")
            print(f"   - Peor modelo: {retrieval_insights.get('worst_model', 'N/A')}")
            print(f"   - Rango Recall@5: {retrieval_insights.get('recall_range', (0, 0))}")
            print(f"   - Desviación estándar Recall: {retrieval_insights.get('recall_std', 0):.3f}")
        
        # Insights de re-ranking
        if 'reranking_insights' in insights and insights['reranking_insights']:
            reranking_insights = insights['reranking_insights']
            print(f"\n🔄 Re-ranking:")
            print(f"   - Mejor modelo: {reranking_insights.get('best_reranking_model', 'N/A')}")
            print(f"   - Efectividad general: {reranking_insights.get('overall_effectiveness', 0):.1%}")
            
            if 'model_stats' in reranking_insights:
                print(f"   - Estadísticas por modelo:")
                for model, stats in reranking_insights['model_stats'].items():
                    print(f"     * {model}: {stats['mean_improvement']:+.3f} ± {stats['std_improvement']:.3f}")
        
        # Insights de calidad
        if 'quality_insights' in insights and insights['quality_insights']:
            quality_insights = insights['quality_insights']
            print(f"\n🎯 Calidad:")
            print(f"   - Mejor combinación: {quality_insights.get('best_quality_combo', 'N/A')}")
            print(f"   - Calidad promedio: {quality_insights.get('overall_quality_mean', 0):.1f}/5")
            
            if 'quality_stats' in quality_insights:
                print(f"   - Estadísticas por métrica:")
                for metric, stats in quality_insights['quality_stats'].items():
                    metric_name = metric.replace('quality_', '').title()
                    print(f"     * {metric_name}: {stats['mean']:.1f} ± {stats['std']:.1f}")
    
    # 3. Análisis de Correlaciones
    print("\n📊 ANÁLISIS DE CORRELACIONES")
    print("-" * 50)
    
    if 'insights' in global_analysis and 'correlation_insights' in global_analysis['insights']:
        correlations = global_analysis['insights']['correlation_insights']
        
        print(f"🔗 Correlaciones entre métricas:")
        for correlation_name, value in correlations.items():
            correlation_display = correlation_name.replace('_', ' vs ').title()
            strength = "Fuerte" if abs(value) > 0.7 else "Moderada" if abs(value) > 0.3 else "Débil"
            direction = "Positiva" if value > 0 else "Negativa"
            print(f"   - {correlation_display}: {value:.3f} ({strength} {direction})")
    
    # 4. Recomendaciones
    print("\n💡 RECOMENDACIONES")
    print("-" * 50)
    
    if 'insights' in global_analysis and 'recommendations' in global_analysis['insights']:
        recommendations = global_analysis['insights']['recommendations']
        
        print(f"🎯 Configuraciones Recomendadas:")
        if recommendations.get('best_overall_config'):
            print(f"   - Mejor configuración general: {recommendations['best_overall_config']}")
        if recommendations.get('best_retrieval_config'):
            print(f"   - Mejor configuración de retrieval: {recommendations['best_retrieval_config']}")
        if recommendations.get('best_quality_config'):
            print(f"   - Mejor configuración de calidad: {recommendations['best_quality_config']}")
        
        print(f"\n🔄 Re-ranking:")
        if recommendations.get('reranking_recommendation'):
            print(f"   - {recommendations['reranking_recommendation']}")
        
        print(f"\n🚀 Sugerencias de Mejora:")
        if recommendations.get('improvement_suggestions'):
            for i, suggestion in enumerate(recommendations['improvement_suggestions'], 1):
                print(f"   {i}. {suggestion}")
        else:
            print("   - No se identificaron mejoras específicas necesarias")
    
    # 5. Resumen Ejecutivo
    print("\n📋 RESUMEN EJECUTIVO")
    print("-" * 50)
    
    # Contar total de combinaciones evaluadas
    total_combinations = 0
    if 'complete_pipeline' in global_analysis:
        total_combinations = len(global_analysis['complete_pipeline'])
    
    print(f"📊 Evaluación completada:")
    print(f"   - Total de combinaciones evaluadas: {total_combinations}")
    print(f"   - Modelos de embedding: {len(EMBEDDING_MODELS)}")
    print(f"   - Modelos de re-ranking: {len(RERANKING_MODELS)}")
    print(f"   - Consultas evaluadas: {len(dataset)}")
    
    # Mejor configuración general
    if 'global_rankings' in global_analysis and 'best_complete_pipeline' in global_analysis['global_rankings']:
        best_config = global_analysis['global_rankings']['best_complete_pipeline'][0]
        print(f"\n🏆 Recomendación Final:")
        print(f"   - Usar configuración: {best_config}")
        print(f"   - Esta configuración ofrece el mejor balance entre retrieval y calidad de respuesta")
    
    print(f"\n✅ Análisis comparativo global completado exitosamente")

# Ejecutar reporte comprensivo
generate_comprehensive_report(global_analysis)


In [ ]:
# Guardar resultados del análisis global
def save_global_analysis_results(global_analysis: Dict[str, Any], 
                                evaluation_name: str = "global_comparative_analysis") -> str:
    """
    Guarda los resultados del análisis comparativo global
    
    Args:
        global_analysis: Resultados del análisis global
        evaluation_name: Nombre de la evaluación
    
    Returns:
        Ruta del archivo guardado
    """
    # Preparar configuración
    config = {
        'embedding_models': list(EMBEDDING_MODELS.keys()),
        'reranking_models': list(RERANKING_MODELS.keys()),
        'retrieval_metrics': RETRIEVAL_METRICS,
        'llm_evaluation_criteria': LLM_EVALUATION_CRITERIA,
        'qdrant_config': QDRANT_CONFIG,
        'llm_config': LLM_CONFIG,
        'dataset_size': len(dataset)
    }
    
    # Preparar metadatos
    metadata = {
        'evaluation_type': 'global_comparative_analysis',
        'description': 'Análisis comparativo global de todas las evaluaciones del pipeline RAG',
        'timestamp': datetime.now().isoformat(),
        'total_queries': len(dataset),
        'total_combinations': len(global_analysis.get('complete_pipeline', {})),
        'analysis_components': ['retrieval', 'reranking', 'complete_pipeline', 'rankings', 'insights']
    }
    
    # Guardar análisis
    filepath = eval_manager.save_evaluation(
        evaluation_name=evaluation_name,
        config=config,
        results={'global_analysis': global_analysis},
        metadata=metadata
    )
    
    print(f"💾 Resultados del análisis global guardados en: {filepath}")
    return filepath

# Guardar resultados del análisis global
global_analysis_filepath = save_global_analysis_results(global_analysis)

# Resumen final del análisis comparativo global
print("\n" + "=" * 80)
print("📋 RESUMEN FINAL - ANÁLISIS COMPARATIVO GLOBAL")
print("=" * 80)

if global_analysis:
    # Estadísticas generales
    print(f"\n📊 Estadísticas Generales:")
    print(f"   - Evaluaciones realizadas: 4 (Retrieval, Re-ranking, Pipeline Completo, Análisis Global)")
    print(f"   - Modelos de embedding evaluados: {len(EMBEDDING_MODELS)}")
    print(f"   - Modelos de re-ranking evaluados: {len(RERANKING_MODELS)}")
    print(f"   - Consultas evaluadas: {len(dataset)}")
    print(f"   - Combinaciones totales: {len(global_analysis.get('complete_pipeline', {}))}")
    
    # Mejores configuraciones identificadas
    if 'global_rankings' in global_analysis:
        rankings = global_analysis['global_rankings']
        
        print(f"\n🏆 Mejores Configuraciones Identificadas:")
        
        if 'best_retrieval' in rankings and rankings['best_retrieval']:
            print(f"   - Mejor Embedding: {rankings['best_retrieval'][0]}")
        
        if 'best_reranking_improvement' in rankings and rankings['best_reranking_improvement']:
            print(f"   - Mejor Re-ranking: {rankings['best_reranking_improvement'][0]}")
        
        if 'best_complete_pipeline' in rankings and rankings['best_complete_pipeline']:
            print(f"   - Mejor Pipeline Completo: {rankings['best_complete_pipeline'][0]}")
        
        if 'best_quality' in rankings and rankings['best_quality']:
            print(f"   - Mejor Calidad: {rankings['best_quality'][0]}")
    
    # Insights clave
    if 'insights' in global_analysis:
        insights = global_analysis['insights']
        
        print(f"\n🔍 Insights Clave:")
        
        # Insight de retrieval
        if 'retrieval_insights' in insights and insights['retrieval_insights']:
            retrieval_insights = insights['retrieval_insights']
            best_model = retrieval_insights.get('best_model', 'N/A')
            recall_range = retrieval_insights.get('recall_range', (0, 0))
            print(f"   - Mejor modelo de embedding: {best_model} (Recall@5: {recall_range[1]:.3f})")
        
        # Insight de re-ranking
        if 'reranking_insights' in insights and insights['reranking_insights']:
            reranking_insights = insights['reranking_insights']
            effectiveness = reranking_insights.get('overall_effectiveness', 0)
            best_reranking = reranking_insights.get('best_reranking_model', 'N/A')
            print(f"   - Efectividad del re-ranking: {effectiveness:.1%} (Mejor: {best_reranking})")
        
        # Insight de calidad
        if 'quality_insights' in insights and insights['quality_insights']:
            quality_insights = insights['quality_insights']
            overall_quality = quality_insights.get('overall_quality_mean', 0)
            best_quality_combo = quality_insights.get('best_quality_combo', 'N/A')
            print(f"   - Calidad promedio: {overall_quality:.1f}/5 (Mejor: {best_quality_combo})")
        
        # Insight de correlaciones
        if 'correlation_insights' in insights and insights['correlation_insights']:
            correlations = insights['correlation_insights']
            retrieval_quality_corr = correlations.get('retrieval_quality', 0)
            print(f"   - Correlación Retrieval-Calidad: {retrieval_quality_corr:.3f}")
    
    # Recomendaciones finales
    if 'insights' in global_analysis and 'recommendations' in global_analysis['insights']:
        recommendations = global_analysis['insights']['recommendations']
        
        print(f"\n💡 Recomendaciones Finales:")
        
        if recommendations.get('best_overall_config'):
            print(f"   - Configuración recomendada: {recommendations['best_overall_config']}")
        
        if recommendations.get('reranking_recommendation'):
            print(f"   - Re-ranking: {recommendations['reranking_recommendation']}")
        
        if recommendations.get('improvement_suggestions'):
            print(f"   - Mejoras sugeridas: {len(recommendations['improvement_suggestions'])} identificadas")
    
    print(f"\n✅ Análisis comparativo global completado exitosamente")
    print(f"📁 Resultados guardados en: {global_analysis_filepath}")
    
    # Próximos pasos sugeridos
    print(f"\n🚀 Próximos Pasos Sugeridos:")
    print(f"   1. Implementar la configuración recomendada en producción")
    print(f"   2. Realizar pruebas A/B con las mejores combinaciones")
    print(f"   3. Monitorear el rendimiento en tiempo real")
    print(f"   4. Iterar y mejorar basándose en feedback de usuarios")
    print(f"   5. Expandir el dataset de evaluación con más consultas")
    
else:
    print("❌ No se pudieron obtener resultados del análisis global")


# 8. Conclusiones

## Resumen Ejecutivo

Este notebook presenta una evaluación comprehensiva del pipeline RAG para búsqueda semántica en derecho laboral paraguayo. A través de 8 secciones estructuradas, se evaluaron múltiples modelos de embedding, re-ranking y generación de respuestas, proporcionando insights valiosos para la optimización del sistema.

### 🎯 Objetivos Cumplidos

1. **Evaluación Sistemática**: Se implementó un framework robusto para evaluar componentes individuales y el pipeline completo
2. **Análisis Comparativo**: Se compararon múltiples configuraciones para identificar las mejores combinaciones
3. **Métricas Objetivas y Subjetivas**: Se combinaron métricas de retrieval con evaluación de calidad usando LLM-as-a-judge
4. **Recomendaciones Accionables**: Se generaron recomendaciones específicas basadas en evidencia empírica

### 📊 Alcance de la Evaluación

- **Modelos de Embedding**: 3 modelos evaluados (sentence-transformers, multilingual, especializados)
- **Modelos de Re-ranking**: 3 modelos cross-encoder evaluados
- **Pipeline Completo**: Evaluación end-to-end con generación y evaluación de respuestas
- **Métricas**: 15+ métricas diferentes (retrieval, calidad, correlaciones)
- **Combinaciones**: Todas las combinaciones posibles evaluadas sistemáticamente


## Hallazgos Clave

### 🏆 Mejores Configuraciones Identificadas

#### 1. Modelos de Embedding
- **Mejor Rendimiento**: [Se actualizará con resultados reales]
- **Características**: [Se actualizará con análisis de patrones]
- **Recomendación**: [Se actualizará con insights específicos]

#### 2. Modelos de Re-ranking
- **Efectividad General**: [Se actualizará con porcentaje de mejoras]
- **Mejor Modelo**: [Se actualizará con modelo recomendado]
- **Impacto en Calidad**: [Se actualizará con análisis de correlaciones]

#### 3. Pipeline Completo
- **Configuración Óptima**: [Se actualizará con mejor combinación]
- **Score Combinado**: [Se actualizará con métrica final]
- **Balance Retrieval-Calidad**: [Se actualizará con análisis de trade-offs]

### 📈 Insights de Rendimiento

#### Correlaciones Identificadas
- **Retrieval vs Calidad**: [Se actualizará con coeficiente de correlación]
- **Coherencia vs Relevancia**: [Se actualizará con análisis de relación]
- **Re-ranking vs Mejoras**: [Se actualizará con efectividad medida]

#### Patrones de Comportamiento
- **Consistencia**: [Se actualizará con análisis de estabilidad]
- **Escalabilidad**: [Se actualizará con rendimiento por consulta]
- **Robustez**: [Se actualizará con análisis de errores]

### 🎯 Métricas de Éxito

#### Retrieval
- **Recall@5 Promedio**: [Se actualizará con valor]
- **Precision@5 Promedio**: [Se actualizará con valor]
- **nDCG@5 Promedio**: [Se actualizará con valor]
- **MRR Promedio**: [Se actualizará con valor]

#### Calidad de Respuestas
- **Score General Promedio**: [Se actualizará con valor]/5
- **Coherencia Promedio**: [Se actualizará con valor]/5
- **Relevancia Promedio**: [Se actualizará con valor]/5
- **Completitud Promedio**: [Se actualizará con valor]/5
- **Fidelidad Promedio**: [Se actualizará con valor]/5
- **Concisión Promedio**: [Se actualizará con valor]/5


In [ ]:
# Generar conclusiones dinámicas basadas en los resultados
def generate_dynamic_conclusions(global_analysis: Dict[str, Any]) -> Dict[str, Any]:
    """
    Genera conclusiones dinámicas basadas en los resultados reales
    """
    conclusions = {
        'best_configurations': {},
        'performance_insights': {},
        'success_metrics': {},
        'recommendations': {},
        'limitations': {},
        'future_work': {}
    }
    
    if not global_analysis:
        return conclusions
    
    # 1. Mejores configuraciones
    if 'global_rankings' in global_analysis:
        rankings = global_analysis['global_rankings']
        
        conclusions['best_configurations'] = {
            'best_embedding': rankings.get('best_retrieval', ['N/A'])[0],
            'best_reranking': rankings.get('best_reranking_improvement', ['N/A'])[0],
            'best_complete_pipeline': rankings.get('best_complete_pipeline', ['N/A'])[0],
            'best_quality': rankings.get('best_quality', ['N/A'])[0]
        }
    
    # 2. Insights de rendimiento
    if 'insights' in global_analysis:
        insights = global_analysis['insights']
        
        # Insights de retrieval
        if 'retrieval_insights' in insights and insights['retrieval_insights']:
            retrieval_insights = insights['retrieval_insights']
            conclusions['performance_insights']['retrieval'] = {
                'best_model': retrieval_insights.get('best_model', 'N/A'),
                'recall_range': retrieval_insights.get('recall_range', (0, 0)),
                'recall_std': retrieval_insights.get('recall_std', 0)
            }
        
        # Insights de re-ranking
        if 'reranking_insights' in insights and insights['reranking_insights']:
            reranking_insights = insights['reranking_insights']
            conclusions['performance_insights']['reranking'] = {
                'best_model': reranking_insights.get('best_reranking_model', 'N/A'),
                'effectiveness': reranking_insights.get('overall_effectiveness', 0)
            }
        
        # Insights de calidad
        if 'quality_insights' in insights and insights['quality_insights']:
            quality_insights = insights['quality_insights']
            conclusions['performance_insights']['quality'] = {
                'best_combo': quality_insights.get('best_quality_combo', 'N/A'),
                'overall_mean': quality_insights.get('overall_quality_mean', 0)
            }
        
        # Correlaciones
        if 'correlation_insights' in insights and insights['correlation_insights']:
            correlations = insights['correlation_insights']
            conclusions['performance_insights']['correlations'] = correlations
    
    # 3. Métricas de éxito
    if 'complete_pipeline' in global_analysis:
        pipeline_data = global_analysis['complete_pipeline']
        
        # Calcular promedios de métricas
        retrieval_metrics = ['retrieval_recall_at_5', 'retrieval_precision_at_5', 
                           'retrieval_ndcg_at_5', 'retrieval_mrr']
        quality_metrics = ['quality_coherence', 'quality_relevance', 'quality_completeness', 
                          'quality_fidelity', 'quality_conciseness', 'quality_overall_score']
        
        conclusions['success_metrics'] = {
            'retrieval': {},
            'quality': {}
        }
        
        # Métricas de retrieval
        for metric in retrieval_metrics:
            values = [data[metric] for data in pipeline_data.values() if metric in data]
            if values:
                conclusions['success_metrics']['retrieval'][metric] = {
                    'mean': np.mean(values),
                    'std': np.std(values)
                }
        
        # Métricas de calidad
        for metric in quality_metrics:
            values = [data[metric] for data in pipeline_data.values() if metric in data]
            if values:
                conclusions['success_metrics']['quality'][metric] = {
                    'mean': np.mean(values),
                    'std': np.std(values)
                }
    
    # 4. Recomendaciones
    if 'insights' in global_analysis and 'recommendations' in global_analysis['insights']:
        recommendations = global_analysis['insights']['recommendations']
        conclusions['recommendations'] = {
            'best_overall_config': recommendations.get('best_overall_config', 'N/A'),
            'reranking_recommendation': recommendations.get('reranking_recommendation', 'N/A'),
            'improvement_suggestions': recommendations.get('improvement_suggestions', [])
        }
    
    # 5. Limitaciones identificadas
    conclusions['limitations'] = {
        'dataset_size': f"Dataset limitado a {len(dataset)} consultas",
        'model_coverage': f"Solo {len(EMBEDDING_MODELS)} modelos de embedding evaluados",
        'domain_specificity': "Evaluación específica para derecho laboral paraguayo",
        'llm_dependency': "Dependencia de LLM para evaluación de calidad",
        'computational_cost': "Evaluación computacionalmente intensiva"
    }
    
    # 6. Trabajo futuro
    conclusions['future_work'] = {
        'dataset_expansion': "Expandir dataset con más consultas y casos edge",
        'model_diversification': "Evaluar más modelos de embedding y re-ranking",
        'domain_generalization': "Probar en otros dominios legales",
        'efficiency_optimization': "Optimizar para menor costo computacional",
        'real_world_validation': "Validar en entorno de producción real"
    }
    
    return conclusions

# Generar conclusiones dinámicas
dynamic_conclusions = generate_dynamic_conclusions(global_analysis)

print("✅ Conclusiones dinámicas generadas basadas en resultados reales")


In [ ]:
# Mostrar conclusiones dinámicas
def display_dynamic_conclusions(conclusions: Dict[str, Any]):
    """
    Muestra las conclusiones dinámicas de forma estructurada
    """
    print("\n" + "=" * 80)
    print("📊 CONCLUSIONES DINÁMICAS - BASADAS EN RESULTADOS REALES")
    print("=" * 80)
    
    # 1. Mejores configuraciones
    if 'best_configurations' in conclusions and conclusions['best_configurations']:
        print("\n🏆 MEJORES CONFIGURACIONES IDENTIFICADAS")
        print("-" * 50)
        
        configs = conclusions['best_configurations']
        print(f"🥇 Mejor Modelo de Embedding: {configs.get('best_embedding', 'N/A')}")
        print(f"🥇 Mejor Modelo de Re-ranking: {configs.get('best_reranking', 'N/A')}")
        print(f"🥇 Mejor Pipeline Completo: {configs.get('best_complete_pipeline', 'N/A')}")
        print(f"🥇 Mejor Calidad de Respuesta: {configs.get('best_quality', 'N/A')}")
    
    # 2. Insights de rendimiento
    if 'performance_insights' in conclusions and conclusions['performance_insights']:
        print("\n📈 INSIGHTS DE RENDIMIENTO")
        print("-" * 50)
        
        insights = conclusions['performance_insights']
        
        # Retrieval insights
        if 'retrieval' in insights:
            retrieval = insights['retrieval']
            print(f"\n📊 Retrieval:")
            print(f"   - Mejor modelo: {retrieval.get('best_model', 'N/A')}")
            print(f"   - Rango Recall@5: {retrieval.get('recall_range', (0, 0))}")
            print(f"   - Desviación estándar: {retrieval.get('recall_std', 0):.3f}")
        
        # Re-ranking insights
        if 'reranking' in insights:
            reranking = insights['reranking']
            print(f"\n🔄 Re-ranking:")
            print(f"   - Mejor modelo: {reranking.get('best_model', 'N/A')}")
            print(f"   - Efectividad: {reranking.get('effectiveness', 0):.1%}")
        
        # Quality insights
        if 'quality' in insights:
            quality = insights['quality']
            print(f"\n🎯 Calidad:")
            print(f"   - Mejor combinación: {quality.get('best_combo', 'N/A')}")
            print(f"   - Calidad promedio: {quality.get('overall_mean', 0):.1f}/5")
        
        # Correlaciones
        if 'correlations' in insights:
            correlations = insights['correlations']
            print(f"\n🔗 Correlaciones:")
            for corr_name, value in correlations.items():
                corr_display = corr_name.replace('_', ' vs ').title()
                strength = "Fuerte" if abs(value) > 0.7 else "Moderada" if abs(value) > 0.3 else "Débil"
                direction = "Positiva" if value > 0 else "Negativa"
                print(f"   - {corr_display}: {value:.3f} ({strength} {direction})")
    
    # 3. Métricas de éxito
    if 'success_metrics' in conclusions and conclusions['success_metrics']:
        print("\n🎯 MÉTRICAS DE ÉXITO")
        print("-" * 50)
        
        metrics = conclusions['success_metrics']
        
        # Métricas de retrieval
        if 'retrieval' in metrics and metrics['retrieval']:
            print(f"\n📊 Retrieval:")
            for metric, stats in metrics['retrieval'].items():
                metric_name = metric.replace('retrieval_', '').replace('_at_5', '@5').title()
                print(f"   - {metric_name}: {stats['mean']:.3f} ± {stats['std']:.3f}")
        
        # Métricas de calidad
        if 'quality' in metrics and metrics['quality']:
            print(f"\n🎯 Calidad:")
            for metric, stats in metrics['quality'].items():
                metric_name = metric.replace('quality_', '').title()
                print(f"   - {metric_name}: {stats['mean']:.1f} ± {stats['std']:.1f}")
    
    # 4. Recomendaciones
    if 'recommendations' in conclusions and conclusions['recommendations']:
        print("\n💡 RECOMENDACIONES")
        print("-" * 50)
        
        recs = conclusions['recommendations']
        print(f"🎯 Configuración Recomendada: {recs.get('best_overall_config', 'N/A')}")
        print(f"🔄 Re-ranking: {recs.get('reranking_recommendation', 'N/A')}")
        
        if recs.get('improvement_suggestions'):
            print(f"\n🚀 Sugerencias de Mejora:")
            for i, suggestion in enumerate(recs['improvement_suggestions'], 1):
                print(f"   {i}. {suggestion}")
    
    # 5. Limitaciones
    if 'limitations' in conclusions and conclusions['limitations']:
        print("\n⚠️ LIMITACIONES IDENTIFICADAS")
        print("-" * 50)
        
        limitations = conclusions['limitations']
        for limitation, description in limitations.items():
            limitation_name = limitation.replace('_', ' ').title()
            print(f"   - {limitation_name}: {description}")
    
    # 6. Trabajo futuro
    if 'future_work' in conclusions and conclusions['future_work']:
        print("\n🚀 TRABAJO FUTURO SUGERIDO")
        print("-" * 50)
        
        future_work = conclusions['future_work']
        for area, description in future_work.items():
            area_name = area.replace('_', ' ').title()
            print(f"   - {area_name}: {description}")

# Mostrar conclusiones dinámicas
display_dynamic_conclusions(dynamic_conclusions)


## Recomendaciones Finales

### 🎯 Configuración Recomendada para Producción

Basándose en los resultados de la evaluación, se recomienda la siguiente configuración:

1. **Modelo de Embedding**: [Se actualizará con el mejor modelo identificado]
2. **Modelo de Re-ranking**: [Se actualizará con recomendación de re-ranking]
3. **Configuración de LLM**: [Se actualizará con configuración óptima]
4. **Parámetros de Qdrant**: [Se actualizará con configuración recomendada]

### 📊 Métricas de Monitoreo

Para el monitoreo en producción, se recomienda seguir estas métricas:

#### Métricas Críticas
- **Recall@5**: Debe mantenerse por encima del [valor] identificado
- **Calidad General**: Debe mantenerse por encima de [valor]/5
- **Tiempo de Respuesta**: Debe ser menor a [valor] segundos

#### Métricas de Calidad
- **Coherencia**: Monitorear degradación en respuestas
- **Relevancia**: Verificar que las respuestas sean pertinentes
- **Fidelidad**: Asegurar que las respuestas sean fieles al contexto

### 🔄 Proceso de Mejora Continua

1. **Monitoreo Semanal**: Revisar métricas de rendimiento
2. **Evaluación Mensual**: Ejecutar evaluación completa con nuevas consultas
3. **Actualización Trimestral**: Re-evaluar modelos y configuraciones
4. **Feedback de Usuarios**: Incorporar feedback real para mejorar métricas

### 🚀 Próximos Pasos

#### Inmediatos (1-2 semanas)
- [ ] Implementar configuración recomendada en producción
- [ ] Configurar monitoreo de métricas críticas
- [ ] Establecer proceso de evaluación continua

#### Corto Plazo (1-3 meses)
- [ ] Expandir dataset de evaluación con más consultas
- [ ] Implementar pruebas A/B con diferentes configuraciones
- [ ] Optimizar para menor costo computacional

#### Mediano Plazo (3-6 meses)
- [ ] Evaluar nuevos modelos de embedding y re-ranking
- [ ] Implementar evaluación automática de calidad
- [ ] Desarrollar dashboard de monitoreo en tiempo real

#### Largo Plazo (6+ meses)
- [ ] Extender evaluación a otros dominios legales
- [ ] Implementar aprendizaje continuo del sistema
- [ ] Desarrollar métricas de satisfacción del usuario


In [ ]:
# Guardar conclusiones finales
def save_final_conclusions(conclusions: Dict[str, Any], 
                          evaluation_name: str = "final_conclusions") -> str:
    """
    Guarda las conclusiones finales del notebook
    """
    # Preparar configuración
    config = {
        'embedding_models': list(EMBEDDING_MODELS.keys()),
        'reranking_models': list(RERANKING_MODELS.keys()),
        'retrieval_metrics': RETRIEVAL_METRICS,
        'llm_evaluation_criteria': LLM_EVALUATION_CRITERIA,
        'qdrant_config': QDRANT_CONFIG,
        'llm_config': LLM_CONFIG,
        'dataset_size': len(dataset)
    }
    
    # Preparar metadatos
    metadata = {
        'evaluation_type': 'final_conclusions',
        'description': 'Conclusiones finales del notebook de evaluación RAG',
        'timestamp': datetime.now().isoformat(),
        'total_queries': len(dataset),
        'notebook_sections': 8,
        'evaluation_components': ['retrieval', 'reranking', 'complete_pipeline', 'global_analysis', 'conclusions']
    }
    
    # Guardar conclusiones
    filepath = eval_manager.save_evaluation(
        evaluation_name=evaluation_name,
        config=config,
        results={'final_conclusions': conclusions},
        metadata=metadata
    )
    
    print(f"💾 Conclusiones finales guardadas en: {filepath}")
    return filepath

# Guardar conclusiones finales
final_conclusions_filepath = save_final_conclusions(dynamic_conclusions)

# Resumen final del notebook
print("\n" + "=" * 80)
print("📋 RESUMEN FINAL DEL NOTEBOOK")
print("=" * 80)

print(f"\n✅ NOTEBOOK DE EVALUACIÓN RAG COMPLETADO")
print(f"📊 Secciones implementadas: 8/8")
print(f"🔧 Componentes evaluados: Retrieval, Re-ranking, Pipeline Completo, Análisis Global")
print(f"📈 Métricas implementadas: 15+ métricas diferentes")
print(f"💾 Archivos generados: {len(eval_manager.list_evaluations())} evaluaciones guardadas")

print(f"\n📁 Archivos de resultados:")
print(f"   - Retrieval: {retrieval_filepath}")
print(f"   - Re-ranking: {reranking_filepath}")
print(f"   - Pipeline Completo: {complete_rag_filepath}")
print(f"   - Análisis Global: {global_analysis_filepath}")
print(f"   - Conclusiones: {final_conclusions_filepath}")

print(f"\n🎯 Próximos pasos recomendados:")
print(f"   1. Revisar conclusiones dinámicas generadas")
print(f"   2. Implementar configuración recomendada")
print(f"   3. Configurar monitoreo de métricas")
print(f"   4. Establecer proceso de evaluación continua")

print(f"\n🚀 ¡Evaluación RAG completada exitosamente!")
print(f"   Este notebook proporciona una base sólida para la optimización")
print(f"   y monitoreo continuo del sistema RAG en producción.")


## Anexo: Estructura del Notebook

### 📚 Secciones Implementadas

1. **Introducción**: Objetivos, dataset, tecnologías y sistema de persistencia
2. **Configuración de Modelos**: Diccionarios de configuración para modelos y métricas
3. **Carga del Dataset**: Funciones de carga, validación y visualización
4. **Configuración del Entorno**: EnvironmentManager para setup completo
5. **Evaluación de Retrievers**: Métricas de embedding + Qdrant
6. **Evaluación con Re-ranking**: Mejoras con cross-encoders
7. **Evaluación del Flujo Completo**: Pipeline RAG con LLM-as-a-judge
8. **Análisis Comparativo Global**: Consolidación y recomendaciones

### 🔧 Componentes Técnicos

#### Clases Principales
- `EvaluationManager`: Sistema de persistencia de evaluaciones
- `EnvironmentManager`: Configuración centralizada del entorno
- `RetrieverEvaluator`: Evaluación de modelos de embedding
- `RerankingEvaluator`: Evaluación de modelos de re-ranking
- `RAGPipelineEvaluator`: Evaluación del pipeline completo
- `GlobalComparativeAnalyzer`: Análisis comparativo global

#### Funciones de Utilidad
- Carga y validación de datasets
- Visualización de resultados
- Cálculo de métricas agregadas
- Análisis de correlaciones
- Generación de recomendaciones

### 📊 Métricas Implementadas

#### Retrieval
- Recall@k, Precision@k, nDCG@k, MRR, Hit Rate@k

#### Calidad (LLM-as-a-judge)
- Coherencia, Relevancia, Completitud, Fidelidad, Concisión

#### Análisis
- Correlaciones entre métricas
- Mejoras de re-ranking
- Score combinado (retrieval + calidad)

### 💾 Sistema de Persistencia

- **Formato**: JSON con metadatos completos
- **Ubicación**: `evaluation_results/` (configurable)
- **Estructura**: Configuración + Resultados + Metadatos
- **Versionado**: Timestamp automático
- **Comparación**: Funciones para comparar evaluaciones históricas

### 🚀 Características Avanzadas

- **Evaluación Automática**: Proceso completamente automatizado
- **Visualizaciones Dinámicas**: Gráficos adaptativos según datos
- **Análisis de Correlaciones**: Identificación automática de patrones
- **Recomendaciones Inteligentes**: Basadas en evidencia empírica
- **Monitoreo Continuo**: Framework para evaluación periódica
